# 第二课：一个真实世界的提示

在上一课中，我们讨论了几个关键的提示技巧，并看到了如何单独使用每个技巧的示例。现在，让我们尝试编写一个更大、融合了我们刚刚涵盖的许多技巧的提示。

---

## 我们的提示目标
本课将重点编写一个医疗记录摘要提示，该提示接收冗长的医疗记录，并生成包含重要信息的摘要，以协助医生为即将到来的预约做准备。

### 患者的医疗记录

每位患者的医疗记录大致如下所示：

```txt
患者姓名：Evelyn Thompson
年龄：78
医疗记录：

1985年：诊断为2型糖尿病，开始服用二甲双胍
1992年：出现高血压，处方赖诺普利
1998年：因骨关节炎行全髋关节置换术（右）
2000年：诊断为甲状腺功能减退症，开始服用左甲状腺素
2003年：白内障手术（双眼）
2005年：因心房颤动入院，开始服用华法林
2008年：诊断为维生素B12缺乏症，开始每月注射
2010年：因A1C水平升高增加二甲双胍剂量
2011年：因短暂性脑缺血发作（TIA）入院，治疗方案中增加阿司匹林
2013年：诊断为2期乳腺癌，接受乳房肿瘤切除术和放疗
2014年：开始服用阿那曲唑预防乳腺癌复发
2015年：出现慢性肾病（CKD）3期，调整二甲双胍剂量
2017年：因骨关节炎行全膝关节置换术（左）
2018年：因肺炎住院，接受静脉抗生素治疗
2019年：注意到轻度认知障碍，开始服用多奈哌齐
2020年：因难治性高血压增加赖诺普利剂量
2021年：复发性尿路感染（UTIs），处方低剂量预防性抗生素
2022年：年度乳腺X光检查结果清晰，但eGFR显示肾功能恶化
2023年：活动能力下降，开始物理治疗和家庭健康助理探访
```

我们的最终目标是生成一致的记录摘要，以帮助医护人员为即将到来的预约做准备。每个摘要应包含关键信息，包括：

*   患者姓名
*   患者年龄
*   关键诊断
*   患者处方药物清单
*   近期关注点
*   医生的行动项

上述医疗记录的示例输出可能如下所示：

```txt
姓名：Evelyn Thompson
年龄：78

关键诊断：
- 2型糖尿病（1985年）
- 高血压（1992年）
- 骨关节炎（1998年和2017年髋关节和膝关节置换术）
- 甲状腺功能减退症（2000年）
- 心房颤动（2005年）
- 维生素B12缺乏症（2008年）
- 短暂性脑缺血发作（TIA）（2011年）
- 乳腺癌（2013年）
- 慢性肾病（CKD）3期（2015年）
- 肺炎（2018年）
- 轻度认知障碍（2019年）
- 复发性尿路感染（UTIs）（2021年）

药物：
- 二甲双胍（糖尿病）
- 赖诺普利（高血压）
- 左甲状腺素（甲状腺功能减退症）
- 华法林（心房颤动）
- 阿司匹林（抗血小板药）
- 阿那曲唑（乳腺癌复发预防）
- 多奈哌齐（认知障碍）
- 低剂量预防性抗生素（复发性尿路感染）

其他治疗：
- 全髋关节置换术（1998年）
- 白内障手术（2003年）
- 维生素B12注射（2008年）
- 乳房肿瘤切除术和放疗（乳腺癌，2013年）
- 全膝关节置换术（2017年）
- 物理治疗和家庭健康助理（2023年）

近期关注点：
- 肾功能恶化（2022年eGFR下降）
- 活动能力下降（2023年）

行动项：
- 监测肾功能并根据需要调整药物
- 继续物理治疗和家庭健康支持
- 评估认知能力下降并调整治疗计划
- 解决活动能力问题和跌倒风险
- 确保遵守推荐的癌症筛查
```

### 五份医疗记录

这是一个包含5份医疗记录的Python列表，我们将用它来测试我们的提示：

In [ ]:
patient_records = [
    """
患者姓名：Evelyn Thompson
年龄：78
医疗记录：

1985年：诊断为2型糖尿病，开始服用二甲双胍
1992年：出现高血压，处方赖诺普利
1998年：因骨关节炎行全髋关节置换术（右）
2000年：诊断为甲状腺功能减退症，开始服用左甲状腺素
2003年：白内障手术（双眼）
2005年：因心房颤动入院，开始服用华法林
2008年：诊断为维生素B12缺乏症，开始每月注射
2010年：因A1C水平升高增加二甲双胍剂量
2011年：因短暂性脑缺血发作（TIA）入院，治疗方案中增加阿司匹林
2013年：诊断为2期乳腺癌，接受乳房肿瘤切除术和放疗
2014年：开始服用阿那曲唑预防乳腺癌复发
2015年：出现慢性肾病（CKD）3期，调整二甲双胍剂量
2017年：因骨关节炎行全膝关节置换术（左）
2018年：因肺炎住院，接受静脉抗生素治疗
2019年：注意到轻度认知障碍，开始服用多奈哌齐
2020年：因难治性高血压增加赖诺普利剂量
2021年：复发性尿路感染（UTIs），处方低剂量预防性抗生素
2022年：年度乳腺X光检查结果清晰，但eGFR显示肾功能恶化
2023年：活动能力下降，开始物理治疗和家庭健康助理探访
    """,
    """
患者姓名：Marcus Reyes
年龄：42
医疗记录：

2001年：诊断为广泛性焦虑症（GAD），开始服用帕罗西汀
2003年：诊断为重度抑郁症（MDD），增加安非他酮
2005年：因自杀意念住院，增加认知行为疗法（CBT）
2007年：诊断为注意力缺陷/多动障碍（ADHD），开始服用哌甲酯
2009年：报告帕罗西汀副作用，改用艾司西酞普兰
2012年：诊断为阻塞性睡眠呼吸暂停（OSA），开始CPAP治疗
2014年：诊断为高血压，开始服用氯沙坦
2015年：注意到体重增加，转诊营养师
2016年：诊断为2型糖尿病，开始服用二甲双胍
2017年：因糖尿病酮症酸中毒（DKA）住院，开始胰岛素治疗
2018年：报告噩梦，从安非他酮改用文拉法辛
2019年：诊断为胃食管反流病（GERD），开始服用奥美拉唑
2020年：离婚，增加治疗次数，增加辩证行为疗法（DBT）
2021年：出现足底筋膜炎，处方矫形器和物理治疗
2022年：因恐慌发作入院，误诊为心肌梗死，心脏检查结果阴性
2023年：尝试自杀，住院精神治疗30天
2023年：出院后，开始服用新的抗精神病药（喹硫平）和情绪稳定剂（拉莫三嗪）
2024年：报告情绪和睡眠改善，体重减轻
2024年：A1C水平改善，胰岛素剂量减少
    """,
    """
患者姓名：Lily Chen
年龄：8
医疗记录：

2016年（出生）：34周出生，诊断为法洛四联症（TOF）
  - 立即手术放置分流器以增加肺血流量
2016年（3个月）：超声心动图显示右心室肥大恶化
2017年（8个月）：法洛四联症完全修复术（室间隔缺损修补术、肺动脉瓣置换术、右心室流出道修复术）
2017年（10个月）：出现术后心律失常，开始服用胺碘酮
2018年（14个月）：注意到发育迟缓，转诊早期干预服务
2018年（18个月）：因语言发育迟缓开始言语治疗
2019年（2岁）：诊断为发育不良，开始高热量饮食
2019年（2.5岁）：因精细运动技能迟缓开始职业治疗
2020年（3岁）：心导管检查显示轻度肺动脉瓣狭窄
2020年（3.5岁）：诊断为感觉处理障碍（SPD）
2021年（4岁）：开始融合式学前教育项目，并有IEP（个体化教育计划）
2021年（4.5岁）：因RSV细支气管炎住院，需要短暂氧气支持
2022年（5岁）：超声心动图显示肺动脉瓣狭窄进展，进行球囊瓣膜成形术
2022年（5.5岁）：诊断为注意力缺陷/多动障碍（ADHD），开始行为疗法
2023年（6岁）：因感音神经性听力损失进行人工耳蜗植入手术
2023年（7岁）：开始主流学校学习，并持续IEP支持
2024年（7.5岁）：职业治疗停止，已达到精细运动技能目标
2024年（8岁）：定期心脏随访显示肺动脉瓣功能稳定
2024年（8岁）：言语发育进展良好，持续治疗中
    """,
    """
患者姓名：Jason Tran
年龄：25
医疗记录：

2010年（11岁）：诊断为哮喘，开始使用沙丁胺醇吸入器
2012年（13岁）：第一次橄榄球脑震荡，短暂意识丧失
2013年（14岁）：踢足球时右胫骨骨折，手术固定
2014年（15岁）：第二次脑震荡，导致脑震荡后综合征
  - 症状：头痛、头晕、记忆问题
  - 转诊儿科神经科医生，进行认知康复治疗
2015年（16岁）：出现焦虑和抑郁，开始服用氟西汀
2016年（17岁）：打篮球时左膝前交叉韧带撕裂，重建手术
  - 6个月康复，改进行非接触性运动
2017年（18岁）：高中毕业，获得学术奖学金进入大学
2018年（19岁）：诊断为与运动损伤相关的创伤后应激障碍（PTSD）
  - 开始认知行为疗法（CBT）
2019年（20岁）：尝试重返篮球运动，出现恐慌发作
  - 增加治疗次数，增加暴露疗法
2020年（21岁）：COVID-19大流行，远程学习，报告焦虑增加
  - 开始正念冥想和瑜伽
2021年（22岁）：诊断为睡眠呼吸暂停，开始CPAP治疗
  - 睡眠研究提示脑震荡与睡眠障碍之间存在关联
2022年（23岁）：逐渐恢复低冲击运动（游泳、骑自行车）
  - 报告情绪和睡眠质量改善
2023年（24岁）：大学毕业，开始从事体育分析工作
  - 持续治疗，现在每两周一次
  - 参与青少年脑震荡意识计划的志愿工作
2024年（25岁）：年度体检——哮喘控制良好，心理健康稳定
  - 过去2年无运动相关损伤
  - 正在为第一次半程马拉松训练
    """,
    """
患者姓名：Amira Khan
年龄：36
医疗记录：

2011年：复发性关节疼痛和疲劳，初步诊断为纤维肌痛
  - 开始服用普瑞巴林和物理治疗
2012年：出现持续性皮疹和光敏感
  - 皮肤科医生诊断为皮肤型红斑狼疮（CLE）
  - 开始使用局部皮质类固醇和防晒霜
2013年：主诉严重疲劳、脱发和认知问题（“脑雾”）
  - 血液检查显示抗核抗体（ANA）和抗双链DNA抗体（anti-dsDNA）阳性
  - 诊断为系统性红斑狼疮（SLE）
  - 开始服用羟氯喹和低剂量泼尼松
2014年：因狼疮性肾炎（III型）住院
  - 肾活检证实诊断
  - 开始服用吗替麦考酚酯并增加泼尼松剂量
2015年：出现继发于SLE的间质性肺病（ILD）
  - 开始环磷酰胺冲击疗法
  - 需要家庭氧疗
2016年：诊断为继发性干燥综合征
  - 症状：干眼症、口干症
  - 开始服用毛果芸香碱和人工泪液
2017年：因狼疮性脑炎住院
  - 症状：癫痫发作、意识模糊、记忆力丧失
  - MRI显示脑部炎症
  - 接受大剂量类固醇和利妥昔单抗治疗
2018年：出现髋部股骨头坏死（AVN），类固醇诱导
  - 接受全髋关节置换手术
2019年：开始服用贝利尤单抗以减少发作频率
  - 逐渐减少泼尼松剂量
2020年：COVID-19大流行，因免疫抑制而自我隔离
  - 远程医疗随访，家庭输液
2021年：诊断为类固醇诱导的糖尿病
  - 开始服用二甲双胍和胰岛素
  - 转诊内分泌科医生和营养师
2022年：狼疮发作，关节疼痛和疲劳加剧
  - 调整药物：增加吗替麦考酚酯，增加阿巴西普
2023年：因长期类固醇使用进行心血管筛查
  - 超声心动图显示早期舒张功能障碍迹象
  - 开始服用ACE抑制剂，转诊心脏康复
2024年：所有器官系统均有改善
  - 逐渐减少免疫抑制剂，密切监测
  - 继续物理治疗，瑜伽以保持关节健康
  - 计划怀孕，咨询高危产科医生
    """
]

让我们看其中一个，只是为了确保我们的列表正常工作：

In [ ]:
patient_records[2]

'\nPatient Name: Lily Chen\nAge: 8\nMedical Record:\n\n2016 (Birth): Born at 34 weeks, diagnosed with Tetralogy of Fallot (TOF)\n  - Immediate surgery to place a shunt for increased pulmonary blood flow\n2016 (3 months): Echocardiogram showed worsening right ventricular hypertrophy\n2017 (8 months): Complete repair of TOF (VSD closure, pulmonary valve replacement, RV outflow tract repair)\n2017 (10 months): Developed post-operative arrhythmias, started on amiodarone\n2018 (14 months): Developmental delay noted, referred to early intervention services\n2018 (18 months): Speech therapy initiated for delayed language development\n2019 (2 years): Diagnosed with failure to thrive, started on high-calorie diet\n2019 (2.5 years): Occupational therapy started for fine motor skill delays\n2020 (3 years): Cardiac catheterization showed mild pulmonary stenosis\n2020 (3.5 years): Diagnosed with sensory processing disorder (SPD)\n2021 (4 years): Started integrated preschool program with IEP (Indivi

---

## 我们最初的“糟糕”提示

让我们从一个非常简单的提示开始，它要求Claude为我们生成摘要。一个非常简单的首次尝试可能看起来像这样：

> 我有这份患者医疗记录。您能为我总结一下吗？
>
> {医疗记录在此处}
>
> 我需要这份摘要以便在患者明天就诊前快速回顾。

让我们将其形式化为一个具体的提示，以便与动态插入的医疗记录一起使用：

In [ ]:
initial_prompt = """
我这里有这份病人的医疗记录。你能帮我总结一下吗？

{record}

我需要这份总结，以便在病人明天就诊前快速回顾。
"""

接下来，让我们编写一个简单的函数，它将接受一份医疗记录，将其插入到我们的提示中，并让Claude生成一个摘要：

In [ ]:
from anthropic import Anthropic
from dotenv import load_dotenv

load_dotenv()
client = Anthropic()

def generate_summary_with_bad_prompt(patient_record):
    prompt_with_record = initial_prompt.format(record=patient_record)
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        messages=[{"role": "user", "content": prompt_with_record}]
    )
    print("===============================")
    print(response.content[0].text)

让我们用 patient_records 列表中所有的5份医疗记录来试一试：

In [ ]:
generate_summary_with_bad_prompt(patient_records[0])

```txt
===============================
以下是Evelyn Thompson女士78岁医疗记录的摘要：

慢性病史：
- 2型糖尿病（自1985年起）——服用二甲双胍，2010年增加剂量
- 高血压（自1992年起）——服用赖诺普利，2020年增加剂量
- 甲状腺功能减退症（自2000年起）——服用左甲状腺素
- 心房颤动（自2005年起）——服用华法林
- 维生素B12缺乏症（自2008年起）——每月注射
- 慢性肾病3期（自2015年起）——二甲双胍剂量已调整
- 轻度认知障碍（自2019年起）——服用多奈哌齐

手术史：
- 全髋关节置换术（1998年）——右侧，因骨关节炎
- 白内障手术（2003年）——双眼
- 乳房肿瘤切除术和放疗（2013年）——针对2期乳腺癌
- 全膝关节置换术（2017年）——左侧，因骨关节炎

癌症病史：
- 乳腺癌（2013年）——目前服用阿那曲唑预防复发

近期问题：
- 复发性尿路感染（2021年）——服用预防性抗生素
- 肾功能恶化（2022年eGFR显示）
- 活动能力下降（2023年）——开始物理治疗和家庭健康助理探访

总而言之，这是一位患有多种慢性疾病的老年患者，需要多重用药和密切监测，特别是糖尿病、高血压、肾病和癌症复发。
```

In [ ]:
generate_summary_with_bad_prompt(patient_records[1])

```txt
===============================
以下是Marcus Reyes（42岁）的医疗记录摘要：

心理健康：
- 2001年诊断为广泛性焦虑症，用帕罗西汀治疗
- 2003年诊断为重度抑郁症，增加安非他酮
- 2005年因自杀意念住院，开始认知行为疗法（CBT）
- 2007年诊断为注意力缺陷/多动障碍（ADHD），开始服用哌甲酯
- 因副作用/疗效不佳，药物随时间调整（艾司西酞普兰、文拉法辛、喹硫平、拉莫三嗪）
- 2023年尝试自杀，需要30天住院精神治疗
- 2020年离婚导致治疗次数增加（增加辩证行为疗法）

其他疾病：
- 2012年阻塞性睡眠呼吸暂停，开始CPAP治疗
- 2014年高血压，开始服用氯沙坦
- 2015年体重增加，转诊营养师
- 2016年诊断为2型糖尿病，开始服用二甲双胍
- 2017年因糖尿病酮症酸中毒住院，开始胰岛素治疗
- 2019年诊断为胃食管反流病（GERD），开始服用奥美拉唑
- 2021年足底筋膜炎，处方矫形器和物理治疗
- 2022年恐慌发作，误诊为心脏病发作

近期更新：
- 2024年：情绪和睡眠改善，体重减轻
- 2024年：A1C水平改善，胰岛素剂量减少

该患者有复杂的病史，涉及多种精神疾病，通过药物和治疗进行管理。他还患有糖尿病、高血压和睡眠呼吸暂停等多种慢性病，需要进行管理。
```

In [ ]:
generate_summary_with_bad_prompt(patient_records[2])

```txt
===============================
以下是Lily Chen 8岁医疗记录的摘要：

Lily于2016年，在孕34周时早产，出生时被诊断患有法洛四联症（TOF），这是一种先天性心脏缺陷。她立即接受了分流手术以增加肺血流量，并于2017年8个月大时完成了法洛四联症的完全修复术。术后，她出现了需要药物治疗的心律失常。

大约从1岁开始，Lily出现了发育迟缓——包括发育不良、需要言语治疗的言语/语言迟缓、需要职业治疗的精细运动技能迟缓，并被诊断出患有感觉处理障碍。在学前班时，她开始接受IEP（个体化教育计划）以获得额外支持。

心脏问题持续存在，3岁时出现轻度肺动脉瓣狭窄，5岁时需要进行球囊瓣膜成形术。她还在5.5岁时被诊断出患有注意力缺陷/多动障碍（ADHD），并开始行为疗法。

其他医疗事件包括4.5岁时因RSV细支气管炎住院并需要氧气支持，以及6岁时因感音神经性听力损失进行了人工耳蜗植入手术。

现在Lily 8岁了，她的心脏功能稳定，但仍在继续言语治疗。她已经达到了精细运动技能目标，并在IEP的支持下接受主流学校教育。
```

In [ ]:
generate_summary_with_bad_prompt(patient_records[3])

```txt
===============================
以下是Jason Tran 25岁医疗记录的摘要：

哮喘与运动损伤
- 11岁时诊断为哮喘，处方沙丁胺醇吸入器
- 从13岁开始多次运动脑震荡，伴有脑震荡后综合征
- 青少年时期骨折和膝关节损伤，需要手术

心理健康
- 16岁时出现焦虑和抑郁，开始服用氟西汀
- 19岁时诊断为与运动损伤相关的创伤后应激障碍（PTSD），开始认知行为疗法（CBT）和暴露疗法
- 20岁时尝试重返篮球运动时出现恐慌发作
- 21岁时在COVID-19大流行期间焦虑增加，开始正念练习

睡眠问题
- 22岁时诊断为睡眠呼吸暂停，可能与脑震荡有关，开始CPAP治疗

近期更新
- 23岁时逐渐恢复低冲击运动，情绪/睡眠改善
- 24岁大学毕业，从事体育分析工作
- 持续每两周一次的治疗和青少年脑震荡意识计划
- 25岁时正在为第一次半程马拉松训练，哮喘和心理健康稳定

关键问题
- 哮喘病史
- 多次脑震荡和运动损伤
- 焦虑、抑郁、创伤后应激障碍
- 睡眠呼吸暂停

我已突出显示了关键医疗问题、心理健康状况和近期进展，以便为明天的预约做准备。如果您需要任何澄清或有其他详细信息需要添加，请告诉我。
```

In [ ]:
generate_summary_with_bad_prompt(patient_records[4])

```txt
===============================
以下是Amira Khan女士冗长而复杂的医疗病史摘要：

Amira Khan是一位36岁的女性，长期患有系统性红斑狼疮（SLE）。她于2011年首次出现关节疼痛和疲劳症状。2012年出现皮疹和光敏感后，被诊断为皮肤型红斑狼疮并开始使用局部类固醇。2013年，自身抗体阳性和多系统受累导致了SLE的诊断。

多年来，Amira经历了许多影响多个器官的狼疮并发症：
- 狼疮性肾炎，需要免疫抑制剂治疗（2014年）
- 间质性肺病，需要环磷酰胺和氧气治疗（2015年）
- 继发性干燥综合征（2016年）
- 狼疮性脑炎伴脑部炎症（2017年）
- 髋部股骨头坏死，需要关节置换术（2018年）

她曾接受羟氯喹、吗替麦考酚酯、环磷酰胺、利妥昔单抗、贝利尤单抗等强化治疗，并长期使用皮质类固醇，导致了糖尿病和舒张功能障碍等副作用。尽管治疗面临挑战，但在多学科护理下，她的病情逐渐改善。

目前，她正在服用阿巴西普、吗替麦考酚酯、ACE抑制剂，并尝试逐渐减少皮质类固醇。她继续进行物理治疗，并有意向咨询未来怀孕事宜。对于她复杂的全身性疾病和药物副作用，需要密切监测。
```

**总结**：

如果我们仔细审视从Claude那里获得的摘要，会发现它们参差不齐！有些摘要只是冗长的段落文本。另一些摘要则被分解成项目符号列表。每个摘要的实际内容也截然不同。摘要内容本身不算太糟，但其格式、长度和摘要数据类型不一致，难以进行程序化处理。让我们来改进它！

---

## 改进提示

正如我们在上一课中学到的，有一套清晰的提示技巧可以帮助我们获得更好、更一致的结果，包括：

*   添加清晰的系统提示，为助手设定上下文和角色，例如医疗专业人员或医疗保健数据分析师
*   使用XML标签（例如，`<patient_record>`）结构化输入数据，以明确提示的哪一部分是输入数据
*   提供清晰、具体的指令，说明摘要应侧重于哪些内容，例如诊断、药物和近期健康问题
*   指定摘要所需的格式，例如项目符号列表或包含特定部分的结构化格式
*   包含一个格式良好的摘要示例，以引导助手的回复
*   使用某种输出XML标签来结构化输出，使用户能够快速找到相关信息

**在接下来的课程中，我们将讨论一种特定、细致入微的提示工程方法和提示技巧选择。在本课中，我们将采取“散弹枪”式方法，一次性使用所有这些技巧。**

### 添加系统提示

让我们逐一处理这些建议。我们将首先定义一个系统提示，以设定助手的上下文和角色。根据我们的经验，这是唯一应该放在系统提示中的信息。

对于我们的用例，系统提示应该告诉Claude，它的角色是充当一名医疗专业人员，擅长将冗长的患者病史翻译成简洁的摘要。

以下是一种方法：

In [ ]:
system = """
您是一位经验丰富的医疗专业人员，专长是将复杂的患者病史转化为简洁、可操作的摘要。
您的职责是分析患者记录，识别关键信息，并以清晰、结构化的格式呈现，以协助诊断和治疗计划。
您的摘要对于需要在就诊前快速了解患者病史的繁忙医护人员来说是无价的。
"""

### 结构化输入数据
在使用Claude时，最重要的提示技巧之一是使用XML标签清晰地标记您的输入数据。在我们当前的提示中，我们像这样提供医疗病史作为主要输入：

> 我有这份患者医疗记录。您能为我总结一下吗？
>
> {医疗记录在此处}
>
> 我需要这份摘要以便在患者明天就诊前快速回顾。

一个可以带来显著改进的简单方法是使用XML标签来包裹我们的输入数据。让我们更新提示，使用`<patient_record>`标签来实现这一点。

### 提供清晰的指令
在使用Claude时，提供清晰直接的指令对于获得最佳结果至关重要。就像我们第一次指导人类完成一项任务一样，我们越是直截了当地解释我们想要什么，Claude的回复就会越好、越准确。

在我们最初的“糟糕”提示中，我们有以下指令：

> 我有这份患者医疗记录。您能为我总结一下吗？

这简直是**极其模糊**！我们需要提出一套清晰具体的指令。我们到底希望摘要是什么样子？

假设我们希望摘要包含以下不同类别的信息：

*   患者姓名
*   患者年龄
*   关键诊断列表
*   患者处方药物列表
*   非药物治疗列表（物理治疗、CBT等）
*   近期关注点列表
*   医生重要的行动项列表

同样重要的是，我们要指定我们所追求的精确输出格式，因此我们一定会要求Claude提供项目符号列表，而不是仅仅一个通用的“列表”。

以下是我们提示的更新版本：

```txt
我需要您帮助我们的医生团队总结患者医疗记录。我们明天有一系列复诊预约，医生们需要快速、富有洞察力的摘要来做准备。

每个摘要应按以下顺序包含以下要素：
- 患者姓名
- 患者年龄
- 按时间顺序列出的关键诊断项目符号列表
- 患者处方药物的项目符号列表
- 其他治疗（非药物治疗，如CBT或物理治疗）的项目符号列表
- 简短的近期关注点项目符号列表
- 帮助我们的医生为即将到来的患者就诊做准备的关键行动项项目符号列表
```

### 添加示例

示例是增强Claude性能并引导其生成所需输出的最强大工具之一。通过在提示中提供几个精心设计的示例，我们可以显著提高Claude回复的准确性、一致性和质量。这种技术对于高度详细或需要结构化输出或遵守特定格式的任务尤其有效。

目前，我们的提示完全不包含任何示例。为了保持简洁，我们将只包含一个示例，尽管通常情况下至少包含两个示例会更好。

我们将基于这份医疗病史来构建示例输入和输出：

```txt
患者姓名：Ethan Blackwood
年龄：55
医疗记录：

2010年：年度体检，发现轻度高血压
  - 开始生活方式干预（饮食、运动）
2012年：因失业诊断为中度抑郁症
  - 开始服用舍曲林和认知行为疗法（CBT）
2014年：找到新工作，报告情绪改善
  - 继续服用舍曲林，减少CBT治疗次数
2015年：轻度背痛，诊断为早期椎间盘退行性病变
  - 处方物理治疗和非处方NSAIDs
2016年：高血压恶化，开始服用赖诺普利
2017年：常规结肠镜检查发现良性息肉，检查过程中已切除
2018年：出现持续性咳嗽，胸部X光检查清晰
  - 诊断为胃食管反流病（GERD）
  - 开始服用奥美拉唑
2019年：诊断为阻塞性睡眠呼吸暂停（OSA）
  - 开始CPAP治疗，报告精力水平改善
2020年：COVID-19大流行，居家办公
  - 报告焦虑增加，恢复CBT治疗（远程医疗）
  - COVID-19疫苗接种（Moderna，两剂）
2021年：轻度膝关节疼痛，MRI显示轻微半月板撕裂
  - 建议关节镜手术，患者选择保守治疗
2022年：年度体检显示糖尿病前期（A1C：6.1%）
  - 加强生活方式干预，转诊营养师
- 因GERD症状缓解停用奥美拉唑
2023年：血液检查显示PSA（前列腺特异性抗原）升高
  - 进行前列腺活检，结果为阴性，无癌症
- 膝关节疼痛加重，同意进行关节镜手术
2024年：术后膝关节恢复良好，持续物理治疗
  - A1C水平改善（5.8%），糖尿病前期已解决
  - 压力测试正常，但超声心动图显示轻度左心室肥厚（LVH）
  - 开始服用低剂量ACE抑制剂进行心脏保护
```

根据这个示例输入，一个遵循我们提示规则的结构良好输出可能如下所示：

```txt
姓名：Ethan Blackwood
年龄：55

关键诊断：
- 高血压（2010年）
- 抑郁症（2012年）
- 椎间盘退行性病变（2015年）
- 胃食管反流病（GERD）（2018年）
- 阻塞性睡眠呼吸暂停（OSA）（2019年）
- 糖尿病前期（2022年）
- 半月板撕裂（2021年）
- 左心室肥厚（LVH）（2024年）

药物：
- 舍曲林（抑郁症）
- 赖诺普利（高血压）
- 奥美拉唑（GERD）——2022年已停用
- 低剂量ACE抑制剂（心脏保护——2024年）

其他治疗：
- 认知行为疗法（CBT）（抑郁症）
- 物理治疗（背痛，术后膝关节恢复）
- CPAP治疗（OSA）
- 膝关节关节镜手术（2023年）

近期关注点：
- 膝关节疼痛加重
- PSA升高（2023年）
- 超声心动图显示左心室肥厚（2024年）

行动项：
- 随访术后膝关节恢复和物理治疗
- 监测PSA水平和前列腺健康
- 优化血压和高血压管理
- 评估LVH发现后是否需要进一步心脏检查
```

### 输出XML结构

一种常见的提示策略是要求Claude在其回复中使用XML标签。即使Claude决定给出像“好的，这是您请求的摘要！”这样的开场白或添加额外内容，这也能使我们最终更容易提取出我们关心的精确内容。

在这种情况下，让Claude在`<summary>`标签内生成医疗记录摘要可能是有意义的。一个简单的实现方法是更新我们的示例，向Claude展示我们所需的格式。

目前，我们提示的示例部分如下所示：

```txt
以下是我们希望摘要格式化的示例：

<example>
<patient_record>
患者姓名：Ethan Blackwood
年龄：55
医疗记录：

2010年：年度体检，发现轻度高血压
  - 开始生活方式干预（饮食、运动）
医疗记录在此处截断以保持文本简短...
</patient_record>

您的输出：

姓名：Ethan Blackwood
年龄：55

关键诊断：
- 高血压（2010年）
摘要在此处截断以保持文本简短...
</example>
```

只需一个简单的改动，我们就可以向Claude表明它应该将输出包裹在`<summary>`标签内：

```txt
以下是我们希望摘要格式化的示例：

<example>
<patient_record>
患者姓名：Ethan Blackwood
年龄：55
医疗记录：

2010年：年度体检，发现轻度高血压
  - 开始生活方式干预（饮食、运动）
医疗记录在此处截断以保持文本简短...
</patient_record>

<summary>
姓名：Ethan Blackwood
年龄：55

关键诊断：
- 高血压（2010年）
摘要在此处截断以保持文本简短...
</summary>
</example>
```

让我们更新提示的示例部分以反映这一变化：

### 提示更改回顾
在我们尝试新的提示之前，让我们回顾一下我们所做的更改：

*   我们添加了一个清晰的系统提示，以设定上下文并赋予Claude明确的角色。
*   我们使用XML标签（在本例中是`<patient_record>`）结构化了输入数据，以明确提示的哪一部分是输入数据。
*   我们提供了清晰、具体的指令，说明摘要应侧重于哪些内容，例如诊断、药物和近期健康问题。
*   我们指定了摘要所需的格式。
*   我们添加了一个示例输入和相应的格式良好的摘要，以引导Claude的回复。
*   我们要求Claude在`<summary>`标签内生成摘要。

---

## 测试新提示
让我们用 `patient_records` 列表中相同的5份医疗记录来测试我们新的 `system` 和 `updated_prompt` 提示：

In [ ]:
def generate_summary_with_improved_prompt(patient_record):
    prompt_with_record = updated_prompt.format(record=patient_record) #use our rewritten prompt!
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        system=system, #add in our system prompt!
        messages=[{"role": "user", "content": prompt_with_record}]
    )
    print("===============================")
    print(response.content[0].text)

In [ ]:
generate_summary_with_improved_prompt(patient_records[0])

```txt
===============================
<summary>
姓名：Evelyn Thompson
年龄：78

关键诊断：
- 2型糖尿病（1985年）
- 高血压（1992年）
- 骨关节炎（1998年和2017年髋关节和膝关节置换术）
- 甲状腺功能减退症（2000年）
- 心房颤动（2005年）
- 维生素B12缺乏症（2008年）
- 短暂性脑缺血发作（TIA）（2011年）
- 乳腺癌（2013年）
- 慢性肾病（CKD）3期（2015年）
- 肺炎（2018年）
- 轻度认知障碍（2019年）
- 复发性尿路感染（UTIs）（2021年）

药物：
- 二甲双胍（糖尿病）
- 赖诺普利（高血压）
- 左甲状腺素（甲状腺功能减退症）
- 华法林（心房颤动）
- 阿司匹林（抗血小板药）
- 阿那曲唑（乳腺癌复发预防）
- 多奈哌齐（认知障碍）
- 低剂量预防性抗生素（复发性尿路感染）

其他治疗：
- 全髋关节置换术（1998年）
- 白内障手术（2003年）
- 维生素B12注射（2008年）
- 乳房肿瘤切除术和放疗（乳腺癌，2013年）
- 全膝关节置换术（2017年）
- 物理治疗和家庭健康助理（2023年）

近期关注点：
- 肾功能恶化（2022年eGFR下降）
- 活动能力下降（2023年）

行动项：
- 监测肾功能并根据需要调整药物
- 继续物理治疗和家庭健康支持
- 评估认知能力下降并调整治疗计划
- 解决活动能力问题和跌倒风险
- 确保遵守推荐的癌症筛查
</summary>
```

In [ ]:
generate_summary_with_improved_prompt(patient_records[1])

```txt
===============================
<summary>
姓名：Marcus Reyes
年龄：42

关键诊断：
- 广泛性焦虑症（GAD）（2001年）
- 重度抑郁症（MDD）（2003年）
- 注意力缺陷/多动障碍（ADHD）（2007年）
- 阻塞性睡眠呼吸暂停（OSA）（2012年）
- 高血压（2014年）
- 2型糖尿病（2016年）
- 胃食管反流病（GERD）（2019年）
- 足底筋膜炎（2021年）

药物：
- 帕罗西汀（GAD）——2009年改用艾司西酞普兰
- 安非他酮（MDD）——2018年改用文拉法辛
- 哌甲酯（ADHD）
- 氯沙坦（高血压）
- 二甲双胍（糖尿病）
- 胰岛素治疗（糖尿病）——2017年开始
- 奥美拉唑（GERD）
- 喹硫平（抗精神病药）（2023年）
- 拉莫三嗪（情绪稳定剂）（2023年）

其他治疗：
- 认知行为疗法（CBT）
- 辩证行为疗法（DBT）（2020年）
- CPAP治疗（OSA）
- 矫形器和物理治疗（足底筋膜炎）
- 住院精神治疗（2023年）

近期关注点：
- 尝试自杀（2023年）
- 恐慌发作（2022年）
- 体重增加和糖尿病控制不佳

行动项：
- 密切监测心理健康，确保遵守新的药物治疗方案
- 随访糖尿病管理，进行体重减轻的生活方式干预
- 继续CBT/DBT并评估是否需要增加治疗次数
- 筛查可能影响睡眠、体重、焦虑的潜在药物副作用
</summary>
```

In [ ]:
generate_summary_with_improved_prompt(patient_records[2])

```txt
===============================
<summary>
姓名：Lily Chen
年龄：8

关键诊断：
- 法洛四联症（TOF）（2016年 - 出生）
- 发育迟缓（2018年）
- 发育不良（2019年）
- 感觉处理障碍（SPD）（2020年）
- 注意力缺陷/多动障碍（ADHD）（2022年）
- 感音神经性听力损失（2023年）

药物：
- 胺碘酮（术后心律失常 - 2017年）

其他治疗：
- 法洛四联症手术修复（2017年）
- 早期干预服务（2018年）
- 言语治疗（2018年至今）
- 职业治疗（2019年 - 2024年）
- 融合式学前教育项目与IEP（2021年）
- ADHD行为疗法（2022年）
- 人工耳蜗植入手术（2023年）
- 主流学校教育与IEP支持（2023年至今）
- 肺动脉瓣狭窄球囊瓣膜成形术（2022年）

近期关注点：
- 肺动脉瓣狭窄进展需要干预（2022年）
- 持续的言语发育迟缓

行动项：
- 监测心脏状况、肺动脉瓣功能
- 继续言语和语言治疗
- 评估是否需要持续的ADHD治疗和IEP支持
- 确保在主流教育环境中获得适当的发育支持
</summary>
```

In [ ]:
generate_summary_with_improved_prompt(patient_records[3])

```txt
<summary>
姓名：Jason Tran
年龄：25

关键诊断：
- 哮喘（2010年）
- 脑震荡（2012年，2014年）
- 脑震荡后综合征（2014年）
- 抑郁和焦虑（2015年）
- 前交叉韧带撕裂（2016年）
- 创伤后应激障碍（PTSD）（2018年）
- 睡眠呼吸暂停（2021年）

药物：
- 沙丁胺醇吸入器（哮喘）
- 氟西汀（抑郁，焦虑）

其他治疗：
- 认知康复治疗（脑震荡后综合征）
- 认知行为疗法（CBT）（PTSD，焦虑）
- 暴露疗法（PTSD）
- 正念冥想和瑜伽（焦虑）
- CPAP治疗（睡眠呼吸暂停）
- 手术固定（胫骨骨折，2013年）
- 前交叉韧带重建手术（2016年）
- 物理康复（前交叉韧带术后）

近期关注点：
- 重返运动和管理焦虑/创伤后应激障碍
- 睡眠质量和睡眠呼吸暂停

行动项：
- 监测哮喘控制和药物依从性
- 继续CBT和暴露疗法治疗创伤后应激障碍/焦虑
- 评估睡眠呼吸暂停是否需要持续CPAP治疗
- 鼓励逐步恢复低冲击运动和训练
- 支持青少年脑震荡意识工作
</summary>
```

In [ ]:
generate_summary_with_improved_prompt(patient_records[4])

```txt
===============================
<summary>
姓名：Amira Khan
年龄：36

关键诊断：
- 纤维肌痛（2011年）
- 皮肤型红斑狼疮（CLE）（2012年）
- 系统性红斑狼疮（SLE）（2013年）
- 狼疮性肾炎（2014年）
- 间质性肺病（ILD）（2015年）
- 干燥综合征（2016年）
- 狼疮性脑炎（2017年）
- 髋部股骨头坏死（AVN）（2018年）
- 类固醇诱导的糖尿病（2021年）
- 舒张功能障碍（2023年）

药物：
- 普瑞巴林（纤维肌痛）
- 羟氯喹（SLE）
- 泼尼松（SLE）
- 吗替麦考酚酯（狼疮性肾炎）
- 环磷酰胺（ILD）
- 毛果芸香碱（干燥综合征）
- 利妥昔单抗（狼疮性脑炎）
- 贝利尤单抗（减少发作频率）
- 二甲双胍和胰岛素（类固醇诱导的糖尿病）
- 阿巴西普（SLE发作）
- ACE抑制剂（舒张功能障碍）

其他治疗：
- 物理治疗（纤维肌痛，关节健康）
- 局部皮质类固醇和防晒霜（CLE）
- 家庭氧疗（ILD）
- 全髋关节置换手术（AVN）
- 瑜伽（关节健康）
- 心脏康复（舒张功能障碍）

近期关注点：
- SLE发作（2022年）
- 早期舒张功能障碍（2023年）
- 计划怀孕（2024年）

行动项：
- 优化SLE管理，监测疾病活动
- 继续逐渐减少免疫抑制剂并密切监测
- 糖尿病和心脏健康的生活方式干预
- 评估生育能力和怀孕计划，咨询高危产科医生
- 监测潜在药物相互作用或致畸性
</summary>
```

**总结**：

它运行得非常棒！每个输出都遵循了我们指定的规则和格式。请向上滚动查看Claude用我们最初的提示生成的输出类型，并将其与我们新的、一致的输出进行比较！

---

## 改变方式：JSON！

我们可能非常乐意让Claude生成当前提示所产生的文本摘要，但通常我们希望获得特定的结构化响应，以便于程序化处理。最常见的方法是使用JSON。

**注意：强制生成JSON响应最简单的方法是通过Claude的工具使用功能，[这在我们工具使用课程的单独一课中有所介绍](https://github.com/anthropics/courses/blob/master/ToolUse/03_structured_outputs.ipynb)。本次演示的目的是展示改变提示所能带来的影响。**

为了调整我们当前的提示以使其生成JSON响应，我们需要进行一些更改：
*   明确告诉Claude我们想要一个JSON结果，并指定JSON输出应包含什么。
*   更新我们的示例摘要，使其成为一个JSON输出。

In [ ]:
# 未更改的系统提示
system = """
您是一位经验丰富的医疗专业人员，擅长将复杂的患者病史转化为简洁、可操作的摘要。
您的职责是分析患者记录，识别关键信息，并以清晰、结构化的格式呈现，以帮助诊断和治疗计划。
您的摘要对于需要在就诊前快速了解患者病史的繁忙医护人员来说是无价的。
"""

# 提示已更新为生成JSON
updated_json_prompt = """
我需要您帮助我们的医生团队总结患者医疗记录。
我们明天有一系列复诊预约，医生们需要快速、有洞察力的摘要来做准备。

请以JSON格式提供这些摘要，结构如下：
{
  "name": "患者全名",
  "age": 患者年龄（整数）,
  "key_diagnoses": [
    {
      "diagnosis": "主要诊断",
      "year": 诊断年份（整数）
    },
    ...
  ],
  "medications": [
    {
      "name": "药物名称",
      "purpose": "简要描述用途"
    },
    ...
  ],
  "other_treatments": [
    {
      "treatment": "治疗名称",
      "purpose": "简要描述用途"
    },
    ...
  ],
  "recent_concerns": [
    "近期健康问题或担忧的简要陈述"
  ],
  "action_items": [
    "待办事项 1",
    "待办事项 2",
    ...
  ]
}

以下是我们希望摘要格式化的示例：

<example>
<patient_record>
患者姓名：Ethan Blackwood
年龄：55
医疗记录：

2010年：年度体检，发现轻度高血压
  - 开始生活方式干预（饮食、运动）
2012年：因失业被诊断为中度抑郁症
  - 开始服用舍曲林并接受认知行为疗法（CBT）
2014年：新工作，报告情绪改善
  - 继续服用舍曲林，减少CBT疗程
2015年：轻度背痛，诊断为早期椎间盘退行性疾病
  - 处方物理治疗和非处方NSAIDs
2016年：高血压恶化，开始服用赖诺普利
2017年：常规结肠镜检查发现良性息肉，已在手术中切除
2018年：出现持续咳嗽，胸部X光片正常
  - 诊断为胃食管反流病（GERD）
  - 开始服用奥美拉唑
2019年：诊断为阻塞性睡眠呼吸暂停（OSA）
  - 开始CPAP治疗，报告精力水平改善
2020年：COVID-19大流行，在家工作
  - 报告焦虑增加，CBT疗程恢复（远程医疗）
  - COVID-19疫苗接种（Moderna，两剂）
2021年：轻度膝盖疼痛，MRI显示轻微半月板撕裂
  - 建议关节镜手术，患者选择保守治疗
2022年：年度体检显示糖尿病前期（A1C：6.1%）
  - 加强生活方式干预，转诊营养师
- 因GERD症状缓解停用奥美拉唑
2023年：血液检查显示PSA（前列腺特异性抗原）升高
  - 进行前列腺活检，结果为阴性，无癌症
- 膝盖疼痛加剧，同意进行关节镜手术
2024年：膝盖术后恢复：良好，继续物理治疗
  - A1C水平改善（5.8%），糖尿病前期已解决
  - 压力测试正常，但超声心动图显示轻度LVH
  - 开始服用低剂量ACE抑制剂进行心脏保护
</patient_record>


<summary>
{
  "name": "Ethan Blackwood",
  "age": 55,
  "key_diagnoses": [
    {
      "diagnosis": "高血压",
      "year": 2010
    },
    {
      "diagnosis": "抑郁症",
      "year": 2012
    },
    {
      "diagnosis": "椎间盘退行性疾病",
      "year": 2015
    },
    {
      "diagnosis": "胃食管反流病（GERD）",
      "year": 2018
    },
    {
      "diagnosis": "阻塞性睡眠呼吸暂停（OSA）",
      "year": 2019
    },
    {
      "diagnosis": "糖尿病前期",
      "year": 2022
    },
    {
      "diagnosis": "左心室肥大（LVH）",
      "year": 2024
    }
  ],
  "medications": [
    {
      "name": "赖诺普利",
      "purpose": "高血压管理"
    },
    {
      "name": "舍曲林",
      "purpose": "抑郁症"
    },
    {
      "name": "ACE抑制剂（低剂量）",
      "purpose": "心脏保护"
    }
  ],
  "other_treatments": [
    {
      "treatment": "认知行为疗法（CBT）",
      "purpose": "抑郁症管理"
    },
    {
      "treatment": "物理治疗",
      "purpose": "背痛和膝盖恢复"
    },
    {
      "treatment": "CPAP治疗",
      "purpose": "阻塞性睡眠呼吸暂停"
    },
    {
      "treatment": "关节镜膝盖手术",
      "purpose": "半月板撕裂修复"
    }
  ],
  "recent_concerns": [
    "膝盖疼痛加剧，已通过手术解决",
    "PSA升高（2023年），活检阴性",
    "轻度左心室肥大（2024年）"
  ],
  "action_items": [
    "跟进膝盖术后恢复和物理治疗进展",
    "监测PSA水平和前列腺健康",
    "评估LVH发现后的心脏健康状况",
    "复查血压管理"
  ]
}
</summary>
</example>

现在，请以相同的格式总结以下患者记录。请将您的JSON摘要输出在<summary>标签内。
"""

# 将这部分分解为单独的变量，以便以后更容易使用`format`方法
# （上述提示变量中的JSON花括号在使用`format`时会导致问题）
medical_record_input_prompt = """
<patient_record>
{record}
</patient_record>
"""

让我们使用以下函数来测试一下：

In [ ]:
def generate_summary_with_json_prompt(patient_record):
    final_prompt_part = medical_record_input_prompt.format(record=patient_record) #add the medical record to the final prompt piece
    complete_prompt = updated_json_prompt + final_prompt_part
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        system=system, #add in our system prompt!
        messages=[{"role": "user", "content": complete_prompt}]
    )
    print(response.content[0].text)

In [ ]:
generate_summary_with_json_prompt(patient_records[0])

<summary>
{
  "name": "Evelyn Thompson",
  "age": 78,
  "key_diagnoses": [
    {
      "diagnosis": "Type 2 Diabetes",
      "year": 1985
    },
    {
      "diagnosis": "Hypertension",
      "year": 1992
    },
    {
      "diagnosis": "Osteoarthritis",
      "year": 1998
    },
    {
      "diagnosis": "Hypothyroidism",
      "year": 2000
    },
    {
      "diagnosis": "Atrial Fibrillation",
      "year": 2005
    },
    {
      "diagnosis": "Vitamin B12 Deficiency",
      "year": 2008
    },
    {
      "diagnosis": "Transient Ischemic Attack (TIA)",
      "year": 2011
    },
    {
      "diagnosis": "Breast Cancer (Stage 2)",
      "year": 2013
    },
    {
      "diagnosis": "Chronic Kidney Disease (CKD Stage 3)",
      "year": 2015
    },
    {
      "diagnosis": "Mild Cognitive Impairment",
      "year": 2019
    }
  ],
  "medications": [
    {
      "name": "Metformin",
      "purpose": "Diabetes management"
    },
    {
      "name": "Lisinopril",
      "purpose": "Hypertensi

太棒了，我们得到了想要的JSON！为了展示我们新的JSON响应是多么容易处理，让我们编写一个函数，为一批患者生成患者摘要，然后输出一位医生一天预约所需的所有“待办事项”列表：

In [ ]:
import re
import json

def generate_summary_with_json_prompt(patient_record):
    final_prompt_part = medical_record_input_prompt.format(record=patient_record) #add the medical record to the final prompt piece
    complete_prompt = updated_json_prompt + final_prompt_part
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        system=system, #add in our system prompt!
        messages=[{"role": "user", "content": complete_prompt}]
    )
    return response.content[0].text

def extract_json_from_summary_tags(summary_string):
     # Use regular expressions to find content between <summary> tags
    match = re.search(r'<summary>\s*(.*?)\s*</summary>', summary_string, re.DOTALL)
    if not match:
        print("No <summary> tags found in the input string.")
        return

    # Extract the JSON string
    return  match.group(1)

def extract_action_items(model_response):
    summary = extract_json_from_summary_tags(model_response)
    try:
        # Parse the JSON string
        summary_data = json.loads(summary)

        # Extract and print action items
        action_items = summary_data.get('action_items', [])
        patient_name = summary_data.get('name', "")
        print(f"Action Items for {patient_name}:")
        for idx, item in enumerate(action_items, 1):
            print(f"{idx}. {item}")
    except json.JSONDecodeError:
        print("Failed to parse JSON. Make sure the content between <summary> tags is valid JSON.")

def generate_daily_action_items(patient_records):
    # Takes a list of patient records, generates a summary for each, and prints the action items:
    for record in patient_records:
        summary = generate_summary_with_json_prompt(record)
        extract_action_items(summary)

In [ ]:
generate_daily_action_items(patient_records)

## 最终输出

```txt
Evelyn Thompson的待办事项：
1.  监测肾功能并根据需要调整药物
2.  评估认知状态并优化治疗
3.  评估活动能力和居家安全，考虑辅助设备
4.  跟进乳腺癌监测和预防

Marcus Reyes的待办事项：
1.  密切监测心理健康，确保有足够的社会支持系统
2.  评估更新药物治疗方案的有效性
3.  继续生活方式和糖尿病管理（饮食、运动）
4.  足底筋膜炎的后续物理治疗
5.  为患者和家属提供心理健康相关资源

Lily Chen的待办事项：
1.  评估当前的教育和治疗需求
2.  监测人工耳蜗植入后的听力和言语进展
3.  评估心脏状况和未来干预的必要性
4.  继续密切的发育监测和支持

Jason Tran的待办事项：
1.  监测哮喘控制和药物依从性
2.  评估心理健康状况和治疗进展
3.  评估睡眠质量和CPAP依从性
4.  鼓励逐步恢复低冲击运动
5.  讨论运动相关头部损伤的可能性

Amira Khan的待办事项：
1.  监测器官系统受累情况并调整治疗
2.  管理类固醇诱导的糖尿病和心血管风险因素
3.  提供孕前咨询并协调高危产科护理
4.  优化物理治疗和生活方式干预
```

# Lesson 2: A real-world prompt

In the previous lesson, we discussed several key prompting tips and saw an example of how to use each in isolation. Let's now try writing a much larger prompt that incorporates many of the techniques we just covered.

---

## Our prompting goal
This lesson will focus on writing a medical record summarizer prompt that takes in long medical records and generates a summary containing important information to assist doctors in preparing for upcoming appointments.

Each patient medical record looks something like this: 

```
Patient Name: Evelyn Thompson
Age: 78
Medical Record:

1985: Diagnosed with type 2 diabetes, started on metformin
1992: Developed hypertension, prescribed lisinopril
1998: Total hip replacement (right) due to osteoarthritis
2000: Diagnosed with hypothyroidism, started on levothyroxine
2003: Cataract surgery (both eyes)
2005: Admitted for atrial fibrillation, started on warfarin
2008: Vitamin B12 deficiency diagnosed, monthly injections started
2010: Increased metformin dose due to rising A1C levels
2011: Admitted for transient ischemic attack (TIA), added aspirin to regimen
2013: Diagnosed with stage 2 breast cancer, underwent lumpectomy and radiation
2014: Started on anastrozole for breast cancer recurrence prevention
2015: Developed chronic kidney disease (CKD) stage 3, metformin adjusted
2017: Total knee replacement (left) due to osteoarthritis
2018: Hospitalized for pneumonia, treated with IV antibiotics
2019: Mild cognitive impairment noted, started on donepezil
2020: Lisinopril dosage increased due to refractory hypertension
2021: Recurrent UTIs, prescribed low-dose prophylactic antibiotics
2022: Annual mammogram clear, but eGFR shows worsening kidney function
2023: Mobility declining, started physical therapy and home health aide visits
```

Our end goal is to generate consistent record summaries to help providers prepare for upcoming appointments.  Each summary should contain key pieces of information including: 

* The patient's name
* The patient's age
* Key diagnoses
* A list of medications the patient is prescribed 
* Recent concerns
* Action items for the provider

 An example output for the above medical record might look something like this: 

```
Name: Evelyn Thompson
Age: 78

Key Diagnoses:
- Type 2 Diabetes (1985)
- Hypertension (1992)
- Osteoarthritis (Hip and Knee Replacements in 1998 and 2017)
- Hypothyroidism (2000)
- Atrial Fibrillation (2005)
- Vitamin B12 Deficiency (2008)
- Transient Ischemic Attack (TIA) (2011)
- Breast Cancer (2013)
- Chronic Kidney Disease (CKD) Stage 3 (2015)
- Pneumonia (2018)
- Mild Cognitive Impairment (2019)
- Recurrent Urinary Tract Infections (UTIs) (2021)

Medications:
- Metformin (Diabetes)
- Lisinopril (Hypertension)
- Levothyroxine (Hypothyroidism)
- Warfarin (Atrial Fibrillation)
- Aspirin (Antiplatelet)
- Anastrozole (Breast Cancer Recurrence Prevention)
- Donepezil (Cognitive Impairment)
- Low-dose Prophylactic Antibiotics (Recurrent UTIs)

Other Treatments:
- Total Hip Replacement (1998)
- Cataract Surgery (2003)
- Vitamin B12 Injections (2008)
- Lumpectomy and Radiation (Breast Cancer, 2013)
- Total Knee Replacement (2017)
- Physical Therapy and Home Health Aide (2023)

Recent Concerns:
- Worsening Kidney Function (eGFR Decline in 2022)
- Declining Mobility (2023)

Action Items:
- Monitor Kidney Function and Adjust Medications as Needed
- Continue Physical Therapy and Home Health Support
- Evaluate for Cognitive Decline and Adjust Treatment Plan
- Address Mobility Issues and Fall Risk
- Ensure Adherence to Recommended Cancer Screening
```

Here's a Python list containing 5 medical records that we'll try our prompt with:

In [7]:
patient_records = [
    """
Patient Name: Evelyn Thompson
Age: 78
Medical Record:

1985: Diagnosed with type 2 diabetes, started on metformin
1992: Developed hypertension, prescribed lisinopril
1998: Total hip replacement (right) due to osteoarthritis
2000: Diagnosed with hypothyroidism, started on levothyroxine
2003: Cataract surgery (both eyes)
2005: Admitted for atrial fibrillation, started on warfarin
2008: Vitamin B12 deficiency diagnosed, monthly injections started
2010: Increased metformin dose due to rising A1C levels
2011: Admitted for transient ischemic attack (TIA), added aspirin to regimen
2013: Diagnosed with stage 2 breast cancer, underwent lumpectomy and radiation
2014: Started on anastrozole for breast cancer recurrence prevention
2015: Developed chronic kidney disease (CKD) stage 3, metformin adjusted
2017: Total knee replacement (left) due to osteoarthritis
2018: Hospitalized for pneumonia, treated with IV antibiotics
2019: Mild cognitive impairment noted, started on donepezil
2020: Lisinopril dosage increased due to refractory hypertension
2021: Recurrent UTIs, prescribed low-dose prophylactic antibiotics
2022: Annual mammogram clear, but eGFR shows worsening kidney function
2023: Mobility declining, started physical therapy and home health aide visits
    """,
    """
Patient Name: Marcus Reyes
Age: 42
Medical Record:

2001: Diagnosed with generalized anxiety disorder (GAD), started on paroxetine
2003: Diagnosed with major depressive disorder (MDD), added bupropion
2005: Hospitalized for suicidal ideation, added cognitive behavioral therapy (CBT)
2007: Diagnosed with attention-deficit/hyperactivity disorder (ADHD), started on methylphenidate
2009: Reported side effects from paroxetine, switched to escitalopram
2012: Diagnosed with obstructive sleep apnea (OSA), started CPAP therapy
2014: Diagnosed with hypertension, started on losartan
2015: Weight gain noted, referred to nutritionist
2016: Diagnosed with type 2 diabetes, started on metformin
2017: Hospitalized for diabetic ketoacidosis (DKA), insulin therapy initiated
2018: Reported nightmares, switched from bupropion to venlafaxine
2019: Gastroesophageal reflux disease (GERD) diagnosis, started on omeprazole
2020: Divorce, increased therapy sessions, added dialectical behavior therapy (DBT)
2021: Developed plantar fasciitis, prescribed orthotics and physical therapy
2022: Admitted for panic attack, mistaken for myocardial infarction, cardiac workup negative
2023: Attempted suicide, inpatient psychiatric treatment for 30 days
2023: Post-discharge, started on new antipsychotic (quetiapine) and mood stabilizer (lamotrigine)
2024: Reports improvement in mood and sleep, weight loss noted
2024: A1C levels improved, insulin dose decreased
    """,
    """
Patient Name: Lily Chen
Age: 8
Medical Record:

2016 (Birth): Born at 34 weeks, diagnosed with Tetralogy of Fallot (TOF)
  - Immediate surgery to place a shunt for increased pulmonary blood flow
2016 (3 months): Echocardiogram showed worsening right ventricular hypertrophy
2017 (8 months): Complete repair of TOF (VSD closure, pulmonary valve replacement, RV outflow tract repair)
2017 (10 months): Developed post-operative arrhythmias, started on amiodarone
2018 (14 months): Developmental delay noted, referred to early intervention services
2018 (18 months): Speech therapy initiated for delayed language development
2019 (2 years): Diagnosed with failure to thrive, started on high-calorie diet
2019 (2.5 years): Occupational therapy started for fine motor skill delays
2020 (3 years): Cardiac catheterization showed mild pulmonary stenosis
2020 (3.5 years): Diagnosed with sensory processing disorder (SPD)
2021 (4 years): Started integrated preschool program with IEP (Individualized Education Plan)
2021 (4.5 years): Hospitalized for RSV bronchiolitis, required brief oxygen support
2022 (5 years): Echocardiogram showed progression of pulmonary stenosis, balloon valvuloplasty performed
2022 (5.5 years): Diagnosed with attention-deficit/hyperactivity disorder (ADHD), started behavioral therapy
2023 (6 years): Cochlear implant surgery for sensorineural hearing loss
2023 (7 years): Started mainstream school with continued IEP support
2024 (7.5 years): Occupational therapy discontinued, met fine motor skill goals
2024 (8 years): Periodic cardiac follow-up shows stable pulmonary valve function
2024 (8 years): Speech development progressing well, ongoing therapy
    """,
    """
Patient Name: Jason Tran
Age: 25
Medical Record:

2010 (11 yrs): Diagnosed with asthma, started on albuterol inhaler
2012 (13 yrs): First football concussion, brief loss of consciousness
2013 (14 yrs): Fractured right tibia during soccer, surgical fixation
2014 (15 yrs): Second concussion, resulting in post-concussion syndrome
  - Symptoms: headaches, dizziness, memory problems
  - Referred to pediatric neurologist, cognitive rehabilitation therapy
2015 (16 yrs): Developed anxiety and depression, started on fluoxetine
2016 (17 yrs): ACL tear (left knee) during basketball, reconstructive surgery
  - 6-month rehabilitation, switched to non-contact sports
2017 (18 yrs): Graduated high school, started college on academic scholarship
2018 (19 yrs): Diagnosed with PTSD related to sports injuries
  - Started cognitive-behavioral therapy (CBT)
2019 (20 yrs): Tried to return to basketball, experienced panic attack
  - Increased therapy sessions, added exposure therapy
2020 (21 yrs): COVID-19 pandemic, remote learning, reported increased anxiety
  - Started mindfulness meditation and yoga
2021 (22 yrs): Diagnosed with sleep apnea, started CPAP therapy
  - Sleep study suggested link between concussions and sleep disorder
2022 (23 yrs): Gradual return to low-impact sports (swimming, cycling)
  - Reported improved mood and sleep quality
2023 (24 yrs): Graduated college, started job in sports analytics
  - Continuing therapy, now biweekly
  - Volunteering with youth concussion awareness program
2024 (25 yrs): Annual check-up - asthma well-controlled, mental health stable
  - No sports-related injuries in past 2 years
  - Training for first half-marathon
    """,
    """
Patient Name: Amira Khan
Age: 36
Medical Record:

2011: Recurrent joint pain and fatigue, initial diagnosis of fibromyalgia
  - Started on pregabalin and physical therapy
2012: Developed persistent rash and photosensitivity
  - Dermatologist diagnosed cutaneous lupus (CLE)
  - Started on topical corticosteroids and sunscreen
2013: Complained of severe fatigue, hair loss, and cognitive issues ("brain fog")
  - Blood tests showed positive ANA and anti-dsDNA antibodies
  - Diagnosed with Systemic Lupus Erythematosus (SLE)
  - Started on hydroxychloroquine and low-dose prednisone
2014: Hospitalized for lupus nephritis (class III)
  - Renal biopsy confirmed diagnosis
  - Started on mycophenolate mofetil and increased prednisone
2015: Developed interstitial lung disease (ILD) secondary to SLE
  - Started on cyclophosphamide pulse therapy
  - Required home oxygen therapy
2016: Diagnosed with secondary Sjögren's syndrome
  - Symptoms: dry eyes, dry mouth
  - Started on pilocarpine and artificial tears
2017: Hospitalized for lupus cerebritis
  - Symptoms: seizures, confusion, memory loss
  - MRI showed brain inflammation
  - Treated with high-dose steroids and rituximab
2018: Developed avascular necrosis (AVN) of hip, steroid-induced
  - Underwent total hip replacement surgery
2019: Started on belimumab to reduce flare frequency
  - Gradual reduction in prednisone dosage
2020: COVID-19 pandemic, self-isolated due to immunosuppression
  - Telemedicine follow-ups, home infusions
2021: Diagnosed with steroid-induced diabetes
  - Started on metformin and insulin
  - Referred to endocrinologist and nutritionist
2022: Flare-up of lupus, increased joint pain and fatigue
  - Adjusted medications: increased mycophenolate, added abatacept
2023: Cardiovascular screening due to long-term steroid use
  - Echo showed early signs of diastolic dysfunction
  - Started on ACE inhibitors, referred to cardio-rehab
2024: Improvement noted in all organ systems
  - Tapering immunosuppressants, monitoring closely
  - Continues physical therapy, yoga for joint health
  - Planning pregnancy, consulted with high-risk OB
    """
]

Let's take a look at one, just to make sure our list working properly:

In [8]:
patient_records[2]

'\nPatient Name: Lily Chen\nAge: 8\nMedical Record:\n\n2016 (Birth): Born at 34 weeks, diagnosed with Tetralogy of Fallot (TOF)\n  - Immediate surgery to place a shunt for increased pulmonary blood flow\n2016 (3 months): Echocardiogram showed worsening right ventricular hypertrophy\n2017 (8 months): Complete repair of TOF (VSD closure, pulmonary valve replacement, RV outflow tract repair)\n2017 (10 months): Developed post-operative arrhythmias, started on amiodarone\n2018 (14 months): Developmental delay noted, referred to early intervention services\n2018 (18 months): Speech therapy initiated for delayed language development\n2019 (2 years): Diagnosed with failure to thrive, started on high-calorie diet\n2019 (2.5 years): Occupational therapy started for fine motor skill delays\n2020 (3 years): Cardiac catheterization showed mild pulmonary stenosis\n2020 (3.5 years): Diagnosed with sensory processing disorder (SPD)\n2021 (4 years): Started integrated preschool program with IEP (Indivi

---

## Our initial "bad" prompt

Let's start with a very simple prompt that asks Claude to generate us a summary. A very simple first attempt might look something like this:

> I have this patient medical record. Can you summarize it for me?
> 
> {medical record goes here}
> 
> I need this for a quick review before the patient's appointment tomorrow.

Let's formalize this into a specific prompt that we can use with a dynamically inserted medical record:

In [10]:
initial_prompt = """
I have this patient medical record. Can you summarize it for me?

{record}

I need this for a quick review before the patient's appointment tomorrow.
"""

Next, let's write a simple function that will accept a single medical record, insert it into our prompt, and get Claude to generate a summary:

In [98]:
from anthropic import Anthropic
from dotenv import load_dotenv

load_dotenv()
client = Anthropic()

def generate_summary_with_bad_prompt(patient_record):
    prompt_with_record = initial_prompt.format(record=patient_record)
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        messages=[{"role": "user", "content": prompt_with_record}]
    )
    print("===============================")
    print(response.content[0].text)

Let's try it will all 5 of the medical records we have in the `patient_records` list:

In [16]:
generate_summary_with_bad_prompt(patient_records[0])

Here is a summary of Evelyn Thompson's 78-year-old medical record:

Chronic Conditions:
- Type 2 diabetes (since 1985) - on metformin, dose increased in 2010
- Hypertension (since 1992) - on lisinopril, dose increased in 2020
- Hypothyroidism (since 2000) - on levothyroxine  
- Atrial fibrillation (since 2005) - on warfarin
- Vitamin B12 deficiency (since 2008) - receiving monthly injections
- Chronic kidney disease stage 3 (since 2015) - metformin adjusted
- Mild cognitive impairment (since 2019) - on donepezil

Surgical History:
- Total hip replacement (1998) - right side, due to osteoarthritis
- Cataract surgery (2003) - both eyes
- Lumpectomy and radiation (2013) - for stage 2 breast cancer  
- Total knee replacement (2017) - left side, due to osteoarthritis

Cancer History: 
- Breast cancer (2013) - currently on anastrozole for recurrence prevention

Recent Issues:
- Recurrent UTIs (2021) - on prophylactic antibiotics
- Worsening kidney function per eGFR (2022)
- Declining mobilit

In [17]:
generate_summary_with_bad_prompt(patient_records[1])

Here's a summary of Marcus Reyes' (age 42) medical record:

Mental Health:
- Diagnosed with generalized anxiety disorder in 2001, treated with paroxetine
- Major depressive disorder diagnosed in 2003, bupropion added
- Hospitalized for suicidal ideation in 2005, started cognitive behavioral therapy (CBT)  
- ADHD diagnosis in 2007, started methylphenidate
- Medications changed over time due to side effects/inefficacy (escitalopram, venlafaxine, quetiapine, lamotrigine)
- Attempted suicide in 2023, required 30-day inpatient psychiatric treatment
- Divorce in 2020 led to increased therapy (added dialectical behavior therapy)

Other Conditions:
- Obstructive sleep apnea in 2012, started CPAP therapy
- Hypertension in 2014, started losartan  
- Weight gain, referred to nutritionist in 2015
- Type 2 diabetes diagnosed in 2016, metformin started
- Hospitalized for diabetic ketoacidosis in 2017, insulin initiated
- GERD diagnosed in 2019, started omeprazole
- Plantar fasciitis in 2021, prescr

In [18]:
generate_summary_with_bad_prompt(patient_records[2])

Here is a summary of Lily Chen's 8-year medical record:

Lily was born prematurely at 34 weeks in 2016 with Tetralogy of Fallot (TOF), a congenital heart defect. She underwent immediate surgery for a shunt and then complete TOF repair at 8 months old in 2017. Post-operatively, she developed arrhythmias requiring medication.

Starting around 1 year old, Lily experienced developmental delays - failure to thrive, speech/language delays requiring therapy, fine motor skill delays requiring occupational therapy, and a sensory processing disorder diagnosis. An IEP was started in preschool for extra support.

Cardiac issues continued with mild pulmonary stenosis at 3 years old, requiring a balloon valvuloplasty at 5 years. She was also diagnosed with ADHD at 5.5 years and started behavioral therapy.

Other medical events included an RSV bronchiolitis hospitalization at 4.5 years requiring oxygen, and cochlear implant surgery at 6 years for sensorineural hearing loss.

Now at 8 years old, Lily 

In [19]:
generate_summary_with_bad_prompt(patient_records[3])

Here is a summary of Jason Tran's 25-year-old medical record:

Asthma & Sports Injuries
- Diagnosed with asthma at age 11, prescribed albuterol inhaler
- Multiple sports concussions starting at age 13, with post-concussion syndrome 
- Fracture and knee injuries requiring surgery in teens

Mental Health
- Developed anxiety and depression at 16, started fluoxetine
- Diagnosed with PTSD at 19 related to sports injuries, started CBT and exposure therapy
- Experienced panic attack at 20 trying to return to basketball
- Increased anxiety during COVID-19 pandemic at 21, started mindfulness practices

Sleep Issues
- Diagnosed with sleep apnea at 22 potentially linked to concussions, started CPAP

Recent Updates
- Gradual return to low-impact sports at 23 with improved mood/sleep
- Graduated college at 24, working in sports analytics 
- Continuing biweekly therapy and youth concussion awareness program
- Training for first half-marathon at 25, asthma and mental health stable

Key Issues
- Histo

In [20]:
generate_summary_with_bad_prompt(patient_records[4])

Here is a summary of Amira Khan's lengthy and complex medical history:

Amira Khan is a 36-year-old woman with a long-standing diagnosis of Systemic Lupus Erythematosus (SLE). Her initial symptoms in 2011 were joint pain and fatigue. After developing rashes and photosensitivity in 2012, she was diagnosed with cutaneous lupus and started on topical steroids. In 2013, positive autoantibodies and multi-system involvement led to the SLE diagnosis.

Over the years, Amira has experienced numerous lupus complications affecting multiple organs:
- Lupus nephritis requiring immunosuppressants (2014)
- Interstitial lung disease requiring cyclophosphamide and oxygen (2015)  
- Secondary Sjogren's syndrome (2016)
- Lupus cerebritis with brain inflammation (2017)
- Avascular necrosis of hip requiring joint replacement (2018)

She has required intensive treatment with hydroxychloroquine, mycophenolate, cyclophosphamide, rituximab, belimumab, and long-term corticosteroids leading to side effects like 

If we take a closer look at the summaries we got from Claude, they're all over the place! Some summaries are just long paragraphs of text. Other summaries are broken down into bulleted lists. The actual content of each summary differs radically. The summary content isn't terrible, but the formatting, length, and types of summary data are inconsistent and would be difficult to use programmatically. Let's improve it!

---

## Improving the prompt

As we learned in the previous lesson, there are a clear set of prompting techniques that could help us get better and more consistent results, including:

* Adding a clear system prompt to set the context and role for the assistant, such as a medical professional or a healthcare data analyst
* Structuring the input data using XML tags (e.g., `<patient_record>`) to make it clear what part of the prompt is the input data
* Provide clear, specific instructions on what the summary should focus on, such as diagnoses, medications, and recent health concerns
* Specify the desired format for the summary, such as bullet points or a structured format with specific sections
* Include an example of a well-formatted summary to guide the assistant's response
* Use some sort of output XML tag to structure the output, making it easy for the user to find the relevant information quickly

**In upcoming lessons, we'll discuss a specific, nuanced approach to prompt engineering and selecting prompting techniques. In this lesson we'll take a "shotgun" approach and use all of them at once.**

### Adding a system prompt
Let's tackle these suggestions one at a time. We'll begin by defining a system prompt to set the context and role for the assistant. In our experience, this is the only information that should go in the system prompt.

For our use case, our system prompt should tell Claude that its role is to act as a medical professional that is talented at translating long patient histories into concise summaries.

Here's one approach:

In [33]:
system = """
You are a highly experienced medical professional with a specialty in translating complex patient histories into concise, actionable summaries. 
Your role is to analyze patient records, identify critical information, and present it in a clear, structured format that aids in diagnosis and treatment planning. 
Your summaries are invaluable for busy healthcare providers who need quick insights into a patient's medical history before appointments.
"""

### Structuring input data
One of the most important prompting tips when working with Claude is to clearly label your input data using XML tags. In our current prompt, we're providing a medical history as the main input like this: 

> I have this patient medical record. Can you summarize it for me?
> 
> {medical record goes here}
> 
> I need this for a quick review before the patient's appointment tomorrow.

A simple addition that can lead to substantial improvement is to use XML tags to wrap our input data.  Let's update our prompt to do this using `<patient_record>` tags

In [34]:
#System Prompt
system = """
You are a highly experienced medical professional with a specialty in translating complex patient histories into concise, actionable summaries. 
Your role is to analyze patient records, identify critical information, and present it in a clear, structured format that aids in diagnosis and treatment planning. 
Your summaries are invaluable for busy healthcare providers who need quick insights into a patient's medical history before appointments.
"""

#Improved Prompt (still a work in progress!)
updated_prompt = """
<patient_record>
{record}
</patient_record>
"""

### Provide clear instructions
When working with Claude, providing clear and direct instructions is essential for achieving the best results. Just as when we instruct a human for the first time on a task, the more we explain exactly what we want in a straightforward manner, the better and more accurate Claude’s response will be.

In our initial "bad" prompt, we had the following instructions:

> I have this patient medical record. Can you summarize it for me?

This is woefully vague! We need to come up with a set of clear and specific instructions. What exactly do we want the summary to look like? 

Let's say that we want the summary to include the following distinct categories of information:

* The patient's name
* The patient's age
* A list of key diagnoses
* A list of medications the patient is prescribed
* A list of non-medication treatments (physical therapy, CBT, etc.)
* A list of recent concerns
* A list of important action items for a physician

​
It's also important that we specify the exact output format that we're after, so we'll make sure to ask Claude for bulleted lists instead of just a generic "list."

Here's an updated version of our prompt:

In [36]:
#System Prompt
system = """
You are a highly experienced medical professional with a specialty in translating complex patient histories into concise, actionable summaries. 
Your role is to analyze patient records, identify critical information, and present it in a clear, structured format that aids in diagnosis and treatment planning. 
Your summaries are invaluable for busy healthcare providers who need quick insights into a patient's medical history before appointments.
"""

#Improved Prompt (still a work in progress!)
updated_prompt = """
I need your help summarizing patient medical records for our team of doctors. 
We have a series of follow-up appointments tomorrow, and the doctors need quick, insightful summaries to prepare. 

Each summary should include the following elements in this order:
- The patient's name
- The patients age
- A bulleted list of key diagnoses in chronological order
- A bulleted list of medications the patient is prescribed
- A bulleted list of other treatments: non-medication treatments like CBT or physical therapy
- A short bulleted list of recent concerns
- A bulleted list of key action items to help our doctors prepare for the upcoming patient visit

<patient_record>
{record}
</patient_record>
"""


### Adding examples

Examples are one of the most powerful tools for enhancing Claude’s performance and guiding it to produce a desired output. By providing a few well-crafted examples in our prompt, we can significantly improve the accuracy, consistency, and quality of Claude’s responses. This technique is particularly effective for tasks that are highly detailed or require structured outputs or adherence to specific formats.

At the moment, our prompt does not include any examples at all.  To keep things brief, we'll include a single example, though it's often better to have at least two.  

We'll base our example input and output on this medical history: 

```
Patient Name: Ethan Blackwood
Age: 55
Medical Record:

2010: Annual check-up, mild hypertension noted
  - Started on lifestyle modifications (diet, exercise)
2012: Diagnosed with moderate depression following job loss
  - Started on sertraline and cognitive-behavioral therapy (CBT)
2014: New job, reported improved mood
  - Continued sertraline, reduced CBT sessions
2015: Mild back pain, diagnosed with early degenerative disc disease
  - Physical therapy and over-the-counter NSAIDs prescribed
2016: Hypertension worsened, started on lisinopril
2017: Routine colonoscopy showed benign polyps, removed during procedure
2018: Developed persistent cough, chest X-ray clear
  - Diagnosed with Gastroesophageal Reflux Disease (GERD)
  - Started on omeprazole
2019: Diagnosed with obstructive sleep apnea (OSA)
  - Started CPAP therapy, reported improved energy levels
2020: COVID-19 pandemic, worked from home
  - Reported increased anxiety, CBT sessions resumed (telehealth)
  - COVID-19 vaccination (Moderna, both doses)
2021: Mild knee pain, MRI showed minor meniscus tear
  - Arthroscopic surgery recommended, patient opted for conservative management
2022: Annual check-up showed pre-diabetes (A1C: 6.1%)
  - Intensified lifestyle modifications, referred to nutritionist
- Discontinued omeprazole due to resolved GERD symptoms
2023: Blood tests showed elevated PSA (Prostate-Specific Antigen)
  - Prostate biopsy performed, results negative for cancer
- Knee pain worsened, agreed to arthroscopic surgery
2024: Post-op knee recovery: good, continuing physical therapy
  - A1C levels improved (5.8%), pre-diabetes resolved
  - Stress test normal, but mild LVH on echocardiogram
  - Started on low-dose ACE inhibitor for cardioprotection
```

Given this example input, a well-structured output that follows our prompt's rules might look like this: 

```
Name: Ethan Blackwood
Age: 55  

Key Diagnoses:
- Hypertension (2010)
- Depression (2012)
- Degenerative Disc Disease (2015) 
- Gastroesophageal Reflux Disease (GERD) (2018)
- Obstructive Sleep Apnea (OSA) (2019)
- Pre-diabetes (2022)
- Meniscus Tear (2021)
- Left Ventricular Hypertrophy (LVH) (2024)

Medications: 
- Sertraline (depression)
- Lisinopril (hypertension)
- Omeprazole (GERD) - discontinued in 2022
- Low-dose ACE inhibitor (cardioprotection - 2024)

Other Treatments:
- Cognitive Behavioral Therapy (CBT) (depression)
- Physical therapy (back pain, post-op knee recovery)
- CPAP therapy (OSA)
- Arthroscopic knee surgery (2023)

Recent Concerns:
- Worsening knee pain
- Elevated PSA (2023) 
- Left ventricular hypertrophy on echocardiogram (2024)

Action Items:  
- Follow up on post-op knee recovery and physical therapy  
- Monitor PSA levels and prostate health
- Optimize blood pressure and hypertension management
- Assess need for further cardiac workup after LVH finding

```

Let's update our prompt to include this example input and output.  To make things extra clear to Claude, we'll make sure to wrap our example inside of `<example>` tags:

让我们更新提示，以包含这个示例输入和输出。为了让Claude更清楚，我们将确保将示例包裹在`<example>`标签内：

In [44]:
#System Prompt
system = """
You are a highly experienced medical professional with a specialty in translating complex patient histories into concise, actionable summaries. 
Your role is to analyze patient records, identify critical information, and present it in a clear, structured format that aids in diagnosis and treatment planning. 
Your summaries are invaluable for busy healthcare providers who need quick insights into a patient's medical history before appointments.
"""

#Improved Prompt (still a work in progress!)
updated_prompt = """
I need your help summarizing patient medical records for our team of doctors. 
We have a series of follow-up appointments tomorrow, and the doctors need quick, insightful summaries to prepare. 

Each summary should include the following elements in this order:
- The patient's name
- The patients age
- A bulleted list of key diagnoses in chronological order
- A bulleted list of medications the patient is prescribed
- A bulleted list of other treatments: non-medication treatments like CBT or physical therapy
- A short bulleted list of recent concerns
- A bulleted list of key action items to help our doctors prepare for the upcoming patient visit

Here's an example of how we'd like the summaries formatted:

<example>
<patient_record>
Patient Name: Ethan Blackwood
Age: 55
Medical Record:

2010: Annual check-up, mild hypertension noted
  - Started on lifestyle modifications (diet, exercise)
2012: Diagnosed with moderate depression following job loss
  - Started on sertraline and cognitive-behavioral therapy (CBT)
2014: New job, reported improved mood
  - Continued sertraline, reduced CBT sessions
2015: Mild back pain, diagnosed with early degenerative disc disease
  - Physical therapy and over-the-counter NSAIDs prescribed
2016: Hypertension worsened, started on lisinopril
2017: Routine colonoscopy showed benign polyps, removed during procedure
2018: Developed persistent cough, chest X-ray clear
  - Diagnosed with Gastroesophageal Reflux Disease (GERD)
  - Started on omeprazole
2019: Diagnosed with obstructive sleep apnea (OSA)
  - Started CPAP therapy, reported improved energy levels
2020: COVID-19 pandemic, worked from home
  - Reported increased anxiety, CBT sessions resumed (telehealth)
  - COVID-19 vaccination (Moderna, both doses)
2021: Mild knee pain, MRI showed minor meniscus tear
  - Arthroscopic surgery recommended, patient opted for conservative management
2022: Annual check-up showed pre-diabetes (A1C: 6.1%)
  - Intensified lifestyle modifications, referred to nutritionist
- Discontinued omeprazole due to resolved GERD symptoms
2023: Blood tests showed elevated PSA (Prostate-Specific Antigen)
  - Prostate biopsy performed, results negative for cancer
- Knee pain worsened, agreed to arthroscopic surgery
2024: Post-op knee recovery: good, continuing physical therapy
  - A1C levels improved (5.8%), pre-diabetes resolved
  - Stress test normal, but mild LVH on echocardiogram
  - Started on low-dose ACE inhibitor for cardioprotection
</patient_record>


Your output: 

Name: Ethan Blackwood
Age: 55  

Key Diagnoses:
- Hypertension (2010)
- Depression (2012)
- Degenerative Disc Disease (2015) 
- Gastroesophageal Reflux Disease (GERD) (2018)
- Obstructive Sleep Apnea (OSA) (2019)
- Pre-diabetes (2022)
- Meniscus Tear (2021)
- Left Ventricular Hypertrophy (LVH) (2024)

Medications: 
- Sertraline (depression)
- Lisinopril (hypertension)
- Omeprazole (GERD) - discontinued in 2022
- Low-dose ACE inhibitor (cardioprotection - 2024)

Other Treatments:
- Cognitive Behavioral Therapy (CBT) (depression)
- Physical therapy (back pain, post-op knee recovery)
- CPAP therapy (OSA)
- Arthroscopic knee surgery (2023)

Recent Concerns:
- Worsening knee pain
- Elevated PSA (2023) 
- Left ventricular hypertrophy on echocardiogram (2024)

Action Items:  
- Follow up on post-op knee recovery and physical therapy  
- Monitor PSA levels and prostate health
- Optimize blood pressure and hypertension management
- Assess need for further cardiac workup after LVH finding
</example>

Now, please summarize the following patient record in the same format:
<patient_record>
{record}
</patient_record>
"""

### Output XML structure

A common prompting strategy is to ask Claude to use XML tags in its responses. This makes it easier to eventually extract the exact content we care about, even if Claude decides to give us a preamble like "Ok, here's the summary you requested!" or adds in additional content. 

In this case, it might make sense to get Claude to generate the medical record summary inside of `<summary>` tags.  One easy implementation involves updating our example update to show Claude our desired format.  

Currently, the example portion of our prompt looks like this: 

```
Here's an example of how we'd like the summaries formatted:

<example>
<patient_record>
Patient Name: Ethan Blackwood
Age: 55
Medical Record:

2010: Annual check-up, mild hypertension noted
  - Started on lifestyle modifications (diet, exercise)
medical record truncated to keep the text short here...
</patient_record>

Your output: 

Name: Ethan Blackwood
Age: 55  

Key Diagnoses:
- Hypertension (2010)
summary truncated to keep the text short here...
</example>
```

With one simple change, we can indicate to Claude that it should wrap the output inside of `<summary>` tags:

```
Here's an example of how we'd like the summaries formatted:

<example>
<patient_record>
Patient Name: Ethan Blackwood
Age: 55
Medical Record:

2010: Annual check-up, mild hypertension noted
  - Started on lifestyle modifications (diet, exercise)
medical record truncated to keep the text short here...
</patient_record>

<summary>
Name: Ethan Blackwood
Age: 55  

Key Diagnoses:
- Hypertension (2010)
summary truncated to keep the text short here...
</summary>
</example>
```

Let's update the example section of our prompt to reflect this change:


In [45]:
#System Prompt
system = """
You are a highly experienced medical professional with a specialty in translating complex patient histories into concise, actionable summaries. 
Your role is to analyze patient records, identify critical information, and present it in a clear, structured format that aids in diagnosis and treatment planning. 
Your summaries are invaluable for busy healthcare providers who need quick insights into a patient's medical history before appointments.
"""

#Improved Prompt (still a work in progress!)
updated_prompt = """
I need your help summarizing patient medical records for our team of doctors. 
We have a series of follow-up appointments tomorrow, and the doctors need quick, insightful summaries to prepare. 

Each summary should include the following elements in this order:
- The patient's name
- The patients age
- A bulleted list of key diagnoses in chronological order
- A bulleted list of medications the patient is prescribed
- A bulleted list of other treatments: non-medication treatments like CBT or physical therapy
- A short bulleted list of recent concerns
- A bulleted list of key action items to help our doctors prepare for the upcoming patient visit

Here's an example of how we'd like the summaries formatted:

<example>
<patient_record>
Patient Name: Ethan Blackwood
Age: 55
Medical Record:

2010: Annual check-up, mild hypertension noted
  - Started on lifestyle modifications (diet, exercise)
2012: Diagnosed with moderate depression following job loss
  - Started on sertraline and cognitive-behavioral therapy (CBT)
2014: New job, reported improved mood
  - Continued sertraline, reduced CBT sessions
2015: Mild back pain, diagnosed with early degenerative disc disease
  - Physical therapy and over-the-counter NSAIDs prescribed
2016: Hypertension worsened, started on lisinopril
2017: Routine colonoscopy showed benign polyps, removed during procedure
2018: Developed persistent cough, chest X-ray clear
  - Diagnosed with Gastroesophageal Reflux Disease (GERD)
  - Started on omeprazole
2019: Diagnosed with obstructive sleep apnea (OSA)
  - Started CPAP therapy, reported improved energy levels
2020: COVID-19 pandemic, worked from home
  - Reported increased anxiety, CBT sessions resumed (telehealth)
  - COVID-19 vaccination (Moderna, both doses)
2021: Mild knee pain, MRI showed minor meniscus tear
  - Arthroscopic surgery recommended, patient opted for conservative management
2022: Annual check-up showed pre-diabetes (A1C: 6.1%)
  - Intensified lifestyle modifications, referred to nutritionist
- Discontinued omeprazole due to resolved GERD symptoms
2023: Blood tests showed elevated PSA (Prostate-Specific Antigen)
  - Prostate biopsy performed, results negative for cancer
- Knee pain worsened, agreed to arthroscopic surgery
2024: Post-op knee recovery: good, continuing physical therapy
  - A1C levels improved (5.8%), pre-diabetes resolved
  - Stress test normal, but mild LVH on echocardiogram
  - Started on low-dose ACE inhibitor for cardioprotection
</patient_record>


<summary>
Name: Ethan Blackwood
Age: 55  

Key Diagnoses:
- Hypertension (2010)
- Depression (2012)
- Degenerative Disc Disease (2015) 
- Gastroesophageal Reflux Disease (GERD) (2018)
- Obstructive Sleep Apnea (OSA) (2019)
- Pre-diabetes (2022)
- Meniscus Tear (2021)
- Left Ventricular Hypertrophy (LVH) (2024)

Medications: 
- Sertraline (depression)
- Lisinopril (hypertension)
- Omeprazole (GERD) - discontinued in 2022
- Low-dose ACE inhibitor (cardioprotection - 2024)

Other Treatments:
- Cognitive Behavioral Therapy (CBT) (depression)
- Physical therapy (back pain, post-op knee recovery)
- CPAP therapy (OSA)
- Arthroscopic knee surgery (2023)

Recent Concerns:
- Worsening knee pain
- Elevated PSA (2023) 
- Left ventricular hypertrophy on echocardiogram (2024)

Action Items:  
- Follow up on post-op knee recovery and physical therapy  
- Monitor PSA levels and prostate health
- Optimize blood pressure and hypertension management
- Assess need for further cardiac workup after LVH finding
</summary>
</example>

Now, please summarize the following patient record in the same format:

<patient_record>
{record}
</patient_record>
"""

### Recap of the prompt changes
Before we try out our new prompt, let's take a look at the changes we made: 

* We added a clear system prompt to set the context and give Claude a clear role.
* We structured the input data using XML tags, `<patient_record>` in our case, to make it clear what part of the prompt is the input data.
* We provided clear, specific instructions on what the summary should focus on, such as diagnoses, medications, and recent health concerns.
* We specified the desired format for the summary.
* We added in an example input and corresponding well-formatted summary to guide Claude's reply.
* We asked Claude to generate the summary inside of `<summary>` tags.

---

## Testing out the new prompt
Let's try using our new `system` and `updated_prompt` prompts with the same 5 medical records in our `patient_records` list:

In [46]:
def generate_summary_with_improved_prompt(patient_record):
    prompt_with_record = updated_prompt.format(record=patient_record) #use our rewritten prompt!
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        system=system, #add in our system prompt!
        messages=[{"role": "user", "content": prompt_with_record}]
    )
    print("===============================")
    print(response.content[0].text)

In [47]:
generate_summary_with_improved_prompt(patient_records[0])

<summary>
Name: Evelyn Thompson
Age: 78

Key Diagnoses:
- Type 2 Diabetes (1985)
- Hypertension (1992)
- Osteoarthritis (Hip and Knee Replacements in 1998 and 2017)
- Hypothyroidism (2000)
- Atrial Fibrillation (2005)
- Vitamin B12 Deficiency (2008)
- Transient Ischemic Attack (TIA) (2011)
- Breast Cancer (2013)
- Chronic Kidney Disease (CKD) Stage 3 (2015)
- Pneumonia (2018)
- Mild Cognitive Impairment (2019)
- Recurrent Urinary Tract Infections (UTIs) (2021)

Medications:
- Metformin (Diabetes)
- Lisinopril (Hypertension)
- Levothyroxine (Hypothyroidism)
- Warfarin (Atrial Fibrillation)
- Aspirin (Antiplatelet)
- Anastrozole (Breast Cancer Recurrence Prevention)
- Donepezil (Cognitive Impairment)
- Low-dose Prophylactic Antibiotics (Recurrent UTIs)

Other Treatments:
- Total Hip Replacement (1998)
- Cataract Surgery (2003)
- Vitamin B12 Injections (2008)
- Lumpectomy and Radiation (Breast Cancer, 2013)
- Total Knee Replacement (2017)
- Physical Therapy and Home Health Aide (2023)

Re

In [48]:
generate_summary_with_improved_prompt(patient_records[1])

<summary>
Name: Marcus Reyes
Age: 42

Key Diagnoses:
- Generalized Anxiety Disorder (GAD) (2001)  
- Major Depressive Disorder (MDD) (2003)
- Attention-Deficit/Hyperactivity Disorder (ADHD) (2007)
- Obstructive Sleep Apnea (OSA) (2012)
- Hypertension (2014)
- Type 2 Diabetes (2016)
- Gastroesophageal Reflux Disease (GERD) (2019)
- Plantar Fasciitis (2021)

Medications:
- Paroxetine (GAD) - switched to escitalopram in 2009
- Bupropion (MDD) - switched to venlafaxine in 2018  
- Methylphenidate (ADHD)
- Losartan (hypertension)
- Metformin (diabetes)
- Insulin therapy (diabetes) - initiated in 2017
- Omeprazole (GERD)  
- Quetiapine (antipsychotic) (2023)
- Lamotrigine (mood stabilizer) (2023)

Other Treatments: 
- Cognitive Behavioral Therapy (CBT)
- Dialectical Behavior Therapy (DBT) (2020)
- CPAP therapy (OSA)
- Orthotics and physical therapy (plantar fasciitis)
- Inpatient psychiatric treatment (2023)

Recent Concerns:
- Suicide attempt (2023)  
- Panic attack (2022)
- Weight gain and

In [49]:
generate_summary_with_improved_prompt(patient_records[2])

<summary>
Name: Lily Chen
Age: 8

Key Diagnoses:
- Tetralogy of Fallot (TOF) (2016 - Birth)
- Developmental Delay (2018)
- Failure to Thrive (2019)  
- Sensory Processing Disorder (SPD) (2020)
- Attention-Deficit/Hyperactivity Disorder (ADHD) (2022)
- Sensorineural Hearing Loss (2023)

Medications:
- Amiodarone (post-operative arrhythmias - 2017)

Other Treatments:
- Surgical Repair of TOF (2017)
- Early Intervention Services (2018) 
- Speech Therapy (2018 - ongoing)
- Occupational Therapy (2019 - 2024)  
- Integrated Preschool Program with IEP (2021)
- Behavioral Therapy for ADHD (2022)
- Cochlear Implant Surgery (2023)
- Mainstream School with IEP Support (2023 - ongoing)
- Balloon Valvuloplasty for Pulmonary Stenosis (2022)

Recent Concerns: 
- Progression of Pulmonary Stenosis requiring intervention (2022)
- Ongoing speech development delays

Action Items:
- Monitor cardiac status, pulmonary valve function
- Continue speech and language therapy  
- Assess need for ongoing ADHD trea

In [50]:
generate_summary_with_improved_prompt(patient_records[3])

<summary>
Name: Jason Tran
Age: 25

Key Diagnoses:
- Asthma (2010)
- Concussions (2012, 2014)
- Post-Concussion Syndrome (2014)
- Depression and Anxiety (2015)
- ACL Tear (2016)
- PTSD (2018)
- Sleep Apnea (2021)

Medications:
- Albuterol inhaler (asthma)
- Fluoxetine (depression, anxiety)

Other Treatments:
- Cognitive Rehabilitation Therapy (post-concussion syndrome)
- Cognitive-Behavioral Therapy (CBT) (PTSD, anxiety)
- Exposure Therapy (PTSD)
- Mindfulness Meditation and Yoga (anxiety)
- CPAP Therapy (sleep apnea)
- Surgical Fixation (fractured tibia, 2013)
- ACL Reconstruction Surgery (2016)
- Physical Rehabilitation (post-ACL surgery)

Recent Concerns:
- Returning to sports and managing anxiety/PTSD
- Sleep quality and sleep apnea

Action Items:
- Monitor asthma control and medication adherence
- Continue CBT and exposure therapy for PTSD/anxiety
- Assess need for ongoing CPAP therapy for sleep apnea
- Encourage gradual return to low-impact sports and training
- Support youth con

In [51]:
generate_summary_with_improved_prompt(patient_records[4])

<summary>
Name: Amira Khan
Age: 36

Key Diagnoses:
- Fibromyalgia (2011)
- Cutaneous Lupus Erythematosus (CLE) (2012)
- Systemic Lupus Erythematosus (SLE) (2013)
- Lupus Nephritis (2014)
- Interstitial Lung Disease (ILD) (2015)
- Sjögren's Syndrome (2016)
- Lupus Cerebritis (2017)
- Avascular Necrosis (AVN) of Hip (2018)
- Steroid-induced Diabetes (2021)
- Diastolic Dysfunction (2023)

Medications:
- Pregabalin (fibromyalgia)
- Hydroxychloroquine (SLE)
- Prednisone (SLE)
- Mycophenolate Mofetil (lupus nephritis)
- Cyclophosphamide (ILD)
- Pilocarpine (Sjögren's syndrome)
- Rituximab (lupus cerebritis)
- Belimumab (reduce flare frequency)
- Metformin and Insulin (steroid-induced diabetes)
- Abatacept (SLE flare)
- ACE Inhibitors (diastolic dysfunction)

Other Treatments:
- Physical Therapy (fibromyalgia, joint health)
- Topical Corticosteroids and Sunscreen (CLE)
- Home Oxygen Therapy (ILD)
- Total Hip Replacement Surgery (AVN)
- Yoga (joint health)
- Cardio-rehab (diastolic dysfunction

It's working great! Each of the outputs follows the rules and format that we specified. 
Scroll back to the top to see the types of outputs Claude generated for us with our initial prompt and compare those to our new consistent outputs!

---

## Switching things up: JSON!

We might be perfectly happy to have Claude generate the text summaries that our current prompt results in, but often we want specific structured responses that are easier to programmatically digest.  The most common approach is to use JSON.  

**Note: The easiest way to 'force' a JSON response is through Claude's tool use functionality, [which we cover in a separate lesson in our tool-use course](https://github.com/anthropics/courses/blob/master/ToolUse/03_structured_outputs.ipynb).  The point of this demonstration is to show the impact that changing a prompt can have.**

To adapt our current prompt so that it generates a JSON response, we need to make a few changes:
* Explicitly tell Claude we want a JSON result and specify what the JSON output should include.
* Update our example summary to be a JSON output.

In [71]:
# Unchanged System Prompt
system = """
You are a highly experienced medical professional with a specialty in translating complex patient histories into concise, actionable summaries. 
Your role is to analyze patient records, identify critical information, and present it in a clear, structured format that aids in diagnosis and treatment planning. 
Your summaries are invaluable for busy healthcare providers who need quick insights into a patient's medical history before appointments.
"""

#Prompt updated to generate JSON
updated_json_prompt = """
I need your help summarizing patient medical records for our team of doctors.
We have a series of follow-up appointments tomorrow, and the doctors need quick, insightful summaries to prepare. 

Please provide these summaries in JSON format with the following structure:
{
  "name": "Patient's full name",
  "age": patient's age as an integer,
  "key_diagnoses": [
    {
      "diagnosis": "Primary diagnosis",
      "year": year of diagnosis as an integer
    },
    ...
  ],
  "medications": [
    {
      "name": "Medication name",
      "purpose": "Brief description of what it's for"
    },
    ...
  ],
  "other_treatments": [
    {
      "treatment": "Treatment name",
      "purpose": "Brief description of what it's for"
    },
    ...
  ],
  "recent_concerns": [
    "Brief statement of recent health issue or concern"
  ],
  "action_items": [
    "Action item 1",
    "Action item 2",
    ...
  ]
}

Here's an example of how we'd like the summaries formatted:

<example>
<patient_record>
Patient Name: Ethan Blackwood
Age: 55
Medical Record:

2010: Annual check-up, mild hypertension noted
  - Started on lifestyle modifications (diet, exercise)
2012: Diagnosed with moderate depression following job loss
  - Started on sertraline and cognitive-behavioral therapy (CBT)
2014: New job, reported improved mood
  - Continued sertraline, reduced CBT sessions
2015: Mild back pain, diagnosed with early degenerative disc disease
  - Physical therapy and over-the-counter NSAIDs prescribed
2016: Hypertension worsened, started on lisinopril
2017: Routine colonoscopy showed benign polyps, removed during procedure
2018: Developed persistent cough, chest X-ray clear
  - Diagnosed with Gastroesophageal Reflux Disease (GERD)
  - Started on omeprazole
2019: Diagnosed with obstructive sleep apnea (OSA)
  - Started CPAP therapy, reported improved energy levels
2020: COVID-19 pandemic, worked from home
  - Reported increased anxiety, CBT sessions resumed (telehealth)
  - COVID-19 vaccination (Moderna, both doses)
2021: Mild knee pain, MRI showed minor meniscus tear
  - Arthroscopic surgery recommended, patient opted for conservative management
2022: Annual check-up showed pre-diabetes (A1C: 6.1%)
  - Intensified lifestyle modifications, referred to nutritionist
- Discontinued omeprazole due to resolved GERD symptoms
2023: Blood tests showed elevated PSA (Prostate-Specific Antigen)
  - Prostate biopsy performed, results negative for cancer
- Knee pain worsened, agreed to arthroscopic surgery
2024: Post-op knee recovery: good, continuing physical therapy
  - A1C levels improved (5.8%), pre-diabetes resolved
  - Stress test normal, but mild LVH on echocardiogram
  - Started on low-dose ACE inhibitor for cardioprotection
</patient_record>


<summary>
{
  "name": "Ethan Blackwood",
  "age": 55,
  "key_diagnoses": [
    {
      "diagnosis": "Hypertension",
      "year": 2010
    },
    {
      "diagnosis": "Depression",
      "year": 2012
    },
    {
      "diagnosis": "Degenerative Disc Disease",
      "year": 2015
    },
    {
      "diagnosis": "Gastroesophageal Reflux Disease (GERD)",
      "year": 2018
    },
    {
      "diagnosis": "Obstructive Sleep Apnea (OSA)",
      "year": 2019
    },
    {
      "diagnosis": "Pre-diabetes",
      "year": 2022
    },
    {
      "diagnosis": "Left Ventricular Hypertrophy (LVH)",
      "year": 2024
    }
  ],
  "medications": [
    {
      "name": "Lisinopril",
      "purpose": "Hypertension management"
    },
    {
      "name": "Sertraline",
      "purpose": "Depression"
    },
    {
      "name": "ACE inhibitor (low-dose)",
      "purpose": "Cardioprotection"
    }
  ],
  "other_treatments": [
    {
      "treatment": "Cognitive Behavioral Therapy (CBT)",
      "purpose": "Depression management"
    },
    {
      "treatment": "Physical therapy",
      "purpose": "Back pain and knee recovery"
    },
    {
      "treatment": "CPAP therapy",
      "purpose": "Obstructive Sleep Apnea"
    },
    {
      "treatment": "Arthroscopic knee surgery",
      "purpose": "Meniscus tear repair"
    }
  ],
  "recent_concerns": [
    "Worsening knee pain, addressed with surgery",
    "Elevated PSA (2023), biopsy negative",
    "Mild left ventricular hypertrophy (2024)"
  ],
  "action_items": [
    "Follow up on post-op knee recovery and PT progress",
    "Monitor PSA levels and prostate health",
    "Assess cardiac health post-LVH finding",
    "Review blood pressure management"
  ]
}
</summary>
</example>

Now, please summarize the following patient record in the same format.  Output your JSON sumary inside of <summary> tags
"""

#Broke this part into its own variable to make it easier to use the `format` method later
# (the JSON curly braces in the above prompt variable cause problems when using `format` )
medical_record_input_prompt = """
<patient_record>
{record}
</patient_record>
"""

Let's test it out using the following function:

In [72]:
def generate_summary_with_json_prompt(patient_record):
    final_prompt_part = medical_record_input_prompt.format(record=patient_record) #add the medical record to the final prompt piece
    complete_prompt = updated_json_prompt + final_prompt_part
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        system=system, #add in our system prompt!
        messages=[{"role": "user", "content": complete_prompt}]
    )
    print(response.content[0].text)

In [73]:
generate_summary_with_json_prompt(patient_records[0])

<summary>
{
  "name": "Evelyn Thompson",
  "age": 78,
  "key_diagnoses": [
    {
      "diagnosis": "Type 2 Diabetes",
      "year": 1985
    },
    {
      "diagnosis": "Hypertension",
      "year": 1992
    },
    {
      "diagnosis": "Osteoarthritis",
      "year": 1998
    },
    {
      "diagnosis": "Hypothyroidism",
      "year": 2000
    },
    {
      "diagnosis": "Atrial Fibrillation",
      "year": 2005
    },
    {
      "diagnosis": "Vitamin B12 Deficiency",
      "year": 2008
    },
    {
      "diagnosis": "Transient Ischemic Attack (TIA)",
      "year": 2011
    },
    {
      "diagnosis": "Breast Cancer (Stage 2)",
      "year": 2013
    },
    {
      "diagnosis": "Chronic Kidney Disease (CKD Stage 3)",
      "year": 2015
    },
    {
      "diagnosis": "Mild Cognitive Impairment",
      "year": 2019
    }
  ],
  "medications": [
    {
      "name": "Metformin",
      "purpose": "Diabetes management"
    },
    {
      "name": "Lisinopril",
      "purpose": "Hypertensi

Great, we're getting the JSON we want! 
To demonstrate how easy our new JSON responses are to work with, let's write a function that generates patient summaries for a batch of patients and then outputs a list of all the "action items" that a given physician might need for a day of appointments:

In [97]:
import re
import json

def generate_summary_with_json_prompt(patient_record):
    final_prompt_part = medical_record_input_prompt.format(record=patient_record) #add the medical record to the final prompt piece
    complete_prompt = updated_json_prompt + final_prompt_part
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        system=system, #add in our system prompt!
        messages=[{"role": "user", "content": complete_prompt}]
    )
    return response.content[0].text

def extract_json_from_summary_tags(summary_string):
     # Use regular expressions to find content between <summary> tags
    match = re.search(r'<summary>\s*(.*?)\s*</summary>', summary_string, re.DOTALL)
    if not match:
        print("No <summary> tags found in the input string.")
        return

    # Extract the JSON string
    return  match.group(1)

def extract_action_items(model_response):
    summary = extract_json_from_summary_tags(model_response)
    try:
        # Parse the JSON string
        summary_data = json.loads(summary)

        # Extract and print action items
        action_items = summary_data.get('action_items', [])
        patient_name = summary_data.get('name', "")
        print(f"Action Items for {patient_name}:")
        for idx, item in enumerate(action_items, 1):
            print(f"{idx}. {item}")
    except json.JSONDecodeError:
        print("Failed to parse JSON. Make sure the content between <summary> tags is valid JSON.")

def generate_daily_action_items(patient_records):
    # Takes a list of patient records, generates a summary for each, and prints the action items:
    for record in patient_records:
        summary = generate_summary_with_json_prompt(record)
        extract_action_items(summary)

In [96]:
generate_daily_action_items(patient_records)

Action Items for Evelyn Thompson:
1. Monitor kidney function and adjust medications as needed
2. Evaluate cognitive status and optimize treatment
3. Assess mobility and home safety, consider assistive devices
4. Follow up on breast cancer surveillance and prevention
Action Items for Marcus Reyes:
1. Monitor mental health closely, ensure adequate support system
2. Evaluate effectiveness of updated medication regimen
3. Continue lifestyle and diabetes management (diet, exercise)
4. Follow-up physical therapy for plantar fasciitis
5. Provide resources for patient and family related to mental health
Action Items for Lily Chen:
1. Evaluate current educational and therapy needs
2. Monitor hearing and speech progress post-cochlear implant
3. Assess cardiac status and need for future interventions
4. Continue close developmental monitoring and support
Action Items for Jason Tran:
1. Monitor asthma control and medication adherence
2. Assess mental health status and therapy progress
3. Evaluate 